In [1]:
#Rating and potential prediction system using current rating, potential and stats, combined with statistics from transfermarkt

In [2]:
#Databases used:
#https://www.kaggle.com/stefanoleone992/fifa-20-complete-player-dataset
#https://www.kaggle.com/kriegsmaschine/soccer-players-values-and-their-statistics
#Merging transfermarkt and sofifa databases

In [3]:
import pandas as pd
pd.options.mode.chained_assignment = None #Disabling chained assigment warning 
import numpy as np
import os
from sklearn.model_selection import train_test_split

from keras.layers import Input, Dense, BatchNormalization, Add, GaussianNoise, Dropout
from keras.models import Model
from keras import regularizers
from keras import optimizers
import re
import tensorflow as tf #tensorflow
from tensorflow import keras #keras
from tensorflow.keras import layers #layers
from keras import backend as K #keras backend
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression


import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from google.colab import drive #library to read google drive files
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
train_df1 ='/content/gdrive/My Drive/Proyects/fifa_deep_learning/players_18.csv' #training data direction
train_df2 ='/content/gdrive/My Drive/Proyects/fifa_deep_learning/transfermarkt_fbref_201718.xlsx' #training data direction
train_df3 ='/content/gdrive/My Drive/Proyects/fifa_deep_learning/players_19.csv' #training data direction
train_df4 ='/content/gdrive/My Drive/Proyects/fifa_deep_learning/transfermarkt_fbref_201819.xlsx' #training data direction
test_df ='/content/gdrive/My Drive/Proyects/fifa_deep_learning/players_20.csv' #validation data direction
test_df2 ='/content/gdrive/My Drive/Proyects/fifa_deep_learning/transfermarkt_fbref_201920.xlsx' #validation data direction
test_df3 ='/content/gdrive/My Drive/Proyects/fifa_deep_learning/players_21.csv' #validation data direction
datos_train1 = pd.read_csv(train_df1) #read training data with pandas
datos_train2 = pd.read_excel(train_df2) #read training data with pandas
datos_train3 = pd.read_csv(train_df3) #read training data with pandas
datos_train4 = pd.read_excel(train_df4) #read training data with pandas
datos_test = pd.read_csv(test_df) #read test data with pandas
datos_test2 = pd.read_excel(test_df2) #read test data with pandas
datos_test_nuevo = pd.read_csv(test_df3) #read test data with pandas
datos_train1

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,...,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,32,1985-02-05,185,80,Portugal,Real Madrid,94,94,95500000,565000,"LW, ST",Right,5,4,5,High/Low,C. Ronaldo,Yes,195800000.0,"#Speedster, #Dribbler, #Distance Shooter, #Acr...",LW,7.0,NaN,2009-07-01,2021.0,LS,7.0,90.0,93.0,82.0,90.0,33.0,80.0,NaN,NaN,NaN,...,63,29,95+1,85,85,95+9,22,31,23,7,11,15,14,11,92+3,92+3,92+3,91+3,92+3,92+3,92+3,91+3,90+3,90+3,90+3,90+3,83+3,83+3,83+3,90+3,66+3,62+3,62+3,62+3,66+3,62+3,53+3,53+3,53+3,62+3
1,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,30,1987-06-24,170,72,Argentina,FC Barcelona,93,93,105000000,565000,RW,Left,5,4,4,Medium/Medium,Messi,Yes,215300000.0,"#Dribbler, #FK Specialist, #Acrobat, #Clinical...",RW,10.0,NaN,2004-07-01,2018.0,RW,10.0,89.0,90.0,86.0,96.0,26.0,61.0,NaN,NaN,NaN,...,48,22,93,90,74,96+2,13,28,26,6,11,15,14,8,89+3,89+3,89+3,92+3,93+3,93+3,93+3,92+3,93+3,93+3,93+3,91+3,84+3,84+3,84+3,91+3,62+3,59+3,59+3,59+3,62+3,58+3,46+3,46+3,46+3,58+3
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar,Neymar da Silva Santos Junior,25,1992-02-05,175,68,Brazil,Paris Saint-Germain,92,94,123000000,280000,LW,Right,5,5,5,High/Medium,Neymar,Yes,236800000.0,"#Speedster, #Dribbler, #Acrobat",LW,10.0,NaN,2017-08-03,2022.0,LW,10.0,92.0,84.0,79.0,95.0,30.0,60.0,NaN,NaN,NaN,...,56,36,90,80,81,92+12,21,24,33,9,9,15,15,11,85+3,85+3,85+3,89+3,89+3,89+3,89+3,89+3,88+3,88+3,88+3,88+3,79+3,79+3,79+3,88+3,64+3,59+3,59+3,59+3,64+3,60+3,47+3,47+3,47+3,60+3
3,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Neuer,31,1986-03-27,193,92,Germany,FC Bayern München,92,92,61000000,230000,GK,Right,5,4,1,Medium/Medium,Normal,Yes,100700000.0,NaN,GK,1.0,NaN,2011-07-01,2021.0,GK,1.0,NaN,NaN,NaN,NaN,NaN,NaN,91.0,90.0,95.0,...,29,30,12,70,47,70,10,10,11,91+2,90,95,91,89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,176580,https://sofifa.com/player/176580/luis-suarez/1...,L. Suárez,Luis Alberto Suárez Díaz,30,1987-01-24,182,86,Uruguay,FC Barcelona,92,92,97000000,510000,ST,Right,5,4,4,High/Medium,Normal,Yes,198900000.0,"#Acrobat, #Clinical Finisher",ST,9.0,NaN,2014-07-11,2021.0,LS,9.0,82.0,90.0,79.0,87.0,42.0,81.0,NaN,NaN,NaN,...,78,41,92,84,85,83,30,45,38,27,25,31,33,37,88+4,88+4,88+4,87+4,89+4,89+4,89+4,87+4,87+4,87+4,87+4,86+4,81+4,81+4,81+4,86+4,68+4,66+4,66+4,66+4,68+4,65+4,59+4,59+4,59+4,65+4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17587,237463,https://sofifa.com/player/237463/adam-kelsey/1...,A. Kelsey,Adam Kelsey,17,1999-11-12,188,74,England,Scunthorpe United,46,63,50000,1000,GK,Right,1,2,1,Medium/Medium,Lean,No,119000.0,NaN,RES,41.0,NaN,2017-01-26,2019.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,47.0,49.0,...,16,9,6,26,17,23,9,11,10,46,47,49-5,42-1,48

In [5]:
#Preprocessing Sofifa Data

In [6]:
#Deleting Non-useful columns
no_use_columns = ['player_url','short_name','value_eur', 	'wage_eur','work_rate','body_type','real_face',
                  'release_clause_eur','player_tags','team_position','team_jersey_number',
                  'loaned_from','joined','contract_valid_until','nation_position','nation_jersey_number',
                  'ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm','rm','lwb',
                  'ldm','cdm','rdm','rwb','lb','lcb','cb','rcb','rb','player_traits','gk_handling','gk_kicking',
                  'gk_positioning','gk_reflexes','gk_diving','gk_speed','pace','shooting','passing','dribbling',
                  'defending','physic','nationality','club','dob','weak_foot', 	'skill_moves','attacking_crossing', 	
                  'attacking_finishing', 	'attacking_heading_accuracy', 'attacking_short_passing', 	'attacking_volleys', 	
                  'skill_dribbling', 	'skill_curve', 	'skill_fk_accuracy','skill_long_passing', 	'skill_ball_control',
                  'movement_acceleration', 	'movement_sprint_speed', 'movement_agility', 	'movement_reactions', 	'movement_balance',
                  'power_shot_power', 	'power_jumping', 'power_stamina', 	'power_strength', 	'power_long_shots', 	'mentality_aggression',
                  'mentality_interceptions','mentality_positioning', 	'mentality_vision', 	'mentality_penalties', 	'mentality_composure',
                  'defending_marking', 'defending_standing_tackle', 	'defending_sliding_tackle', 	'goalkeeping_diving', 	'goalkeeping_handling', 	
                  'goalkeeping_kicking', 	'goalkeeping_positioning', 	'goalkeeping_reflexes']

no_use_columns4 = ['Name','Age', 	'Photo', 	'Nationality', 	'Flag', 	'Potential', 	'Club', 	'Club Logo', 	'Value', 	'Wage',
                   'Special', 	'Preferred Foot', 	'International Reputation', 	'Weak Foot', 	'Skill Moves', 	'Work Rate', 	'Body Type',
                   'Real Face', 	'Position', 	'Jersey Number', 	'Joined', 	'Loaned From', 	'Contract Valid Until', 	'Height', 	'Weight',
                   'Crossing', 	'Finishing', 	'HeadingAccuracy', 	'ShortPassing',	'Volleys', 	'Dribbling',	'Curve', 	'FKAccuracy', 	'LongPassing',
                   'BallControl', 	'Acceleration', 	'SprintSpeed', 	'Agility',	'Reactions', 	'Balance', 	'ShotPower', 	'Jumping', 	'Stamina',
                   'Strength', 	'LongShots', 	'Aggression', 	'Interceptions', 	'Positioning', 	'Vision', 	'Penalties', 	'Composure', 	'Marking',
                   'StandingTackle', 	'SlidingTackle', 	'GKDiving', 	'GKHandling', 	'GKKicking', 	'GKPositioning', 	'GKReflexes', 	'Best Position',	
                   'Best Overall Rating', 	'Release Clause', 	'DefensiveAwareness']

datos_train1 = datos_train1.drop(no_use_columns, axis = 1)
datos_test = datos_test.drop(no_use_columns, axis = 1)
datos_train3 = datos_train3.drop(no_use_columns, axis = 1)
datos_test_nuevo = datos_test_nuevo.drop(no_use_columns4, axis = 1)
datos_train1

,sofifa_id,long_name,age,height_cm,weight_kg,overall,potential,player_positions,preferred_foot,international_reputation
0,20801,Cristiano Ronaldo dos Santos Aveiro,32,185,80,94,94,"LW, ST",Right,5
1,158023,Lionel Andrés Messi Cuccittini,30,170,72,93,93,RW,Left,5
2,190871,Neymar da Silva Santos Junior,25,175,68,92,94,LW,Right,5
3,167495,Manuel Neuer,31,193,92,92,92,GK,Right,5
4,176580,Luis Alberto Suárez Díaz,30,182,86,92,92,ST,Right,5
...,...,...,...,...,...,...,...,...,...,...
17587,237463,Adam Kelsey,17,188,74,46,63,GK,Right,1
17588,231381,Jordan Young,17,175,71,46,61,ST,Left,1
17589,240404,Jack Keeble,18,172,66,46,56,CB,Right,1
17590,11728,Barry Richardson,47,185,77,46,46,GK,Right,1


In [7]:
#Filling NaN values with 0 or droping if in rows, all columns are NaN 
datos_train1 = datos_train1.dropna()
# Numerical imputation for NaN values on datos_train2
datos_train2 = datos_train2.fillna(0)
datos_train3 = datos_train3.dropna()
datos_train4 = datos_train4.dropna()
datos_test = datos_test.dropna()
datos_test2 = datos_test2.dropna()
datos_train2

,Unnamed: 0,player,nationality,position,squad,age,birth_year,value,height,position2,foot,league,games,games_starts,minutes,goals,assists,pens_made,pens_att,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,npxg,xa,xg_per90,xa_per90,xg_xa_per90,npxg_per90,npxg_xa_per90,minutes_90s,shots_total,shots_on_target,shots_free_kicks,shots_on_target_pct,shots_total_per90,...,own_goals_against_gkm,psxg_gkm,psnpxg_per_shot_on_target_againstm,psxg_net_gkm,psxg_net_per90_gkm,passes_completed_launched_gkm,passes_launched_gkm,passes_pct_launched_gkm,passes_gkm,passes_throws_gkm,pct_passes_launched_gkm,passes_length_avg_gkm,goal_kicksm,pct_goal_kicks_launchedm,goal_kick_length_avgm,crosses_gkm,crosses_stopped_gkm,crosses_stopped_pct_gkm,def_actions_outside_pen_area_gkm,def_actions_outside_pen_area_per90_gkm,avg_distance_def_actions_gkm,LgRk,MP,W,D,L,GF,GA,GDiff,Pts,Pts/G,xG,xGA,xGDiff,xGDiff/90,Attendance,CL,WinCL,CLBestScorer,Season
0,379,Burgui,es ESP,"FW,MF",Alavés,23,1993,1800000,186,attack - Left Winger,right,La Liga,23,12,1037,1,1,0,0,1,0,0.09,0.09,0.17,0.09,0.17,0.9,0.9,1.7,0.08,0.15,0.23,0.08,0.23,11.5,14,4,1,28.6,1.22,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14,38,15,2,21,40,50,-10,47,1.24,39.0,53.2,-14.2,-0.37,16.819,0,0,0,201718#
1,2530,Raphaël Varane,fr FRA,DF,Real Madrid,24,1993,70000000,191,Defender - Centre-Back,right,La Liga,27,27,2320,0,1,0,0,3,0,0.00,0.04,0.04,0.00,0.04,0.9,0.9,0.9,0.04,0.04,0.07,0.04,0.07,25.8,10,2,0,20.0,0.39,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3,38,22,10,6,94,44,50,76,2.00,83.3,45.4,37.9,1.00,66161.000,1,1,0,201718#
2,721,Rubén Duarte,es ESP,DF,Alavés,21,1995,2000000,179,Defender - Left-Back,left,La Liga,24,24,2138,0,2,0,0,9,2,0.00,0.08,0.08,0.00,0.08,0.5,0.5,1.6,0.02,0.07,0.09,0.02,0.09,23.8,8,1,0,12.5,0.34,...,0.0,0.020175,0.000094,0.0,0.0,0.097368,0.322515,0.00883,0.254094,0.014327,0.026433,0.018012,0.093275,0.029064,0.020643,0.087427,0.006433,0.002164,0.011111,0.000292,0.004737,14,38,15,2,21,40,50,-10,47,1.24,39.0,53.2,-14.2,-0.37,16.819,0,0,0,201718#
3,2512,Samuel Umtiti,fr FRA,DF,Barcelona,23,1993,60000000,182,Defender - Centre-Back,left,La Liga,25,24,2189,1,0,0,0,7,0,0.04,0.00,0.04,0.04,0.04,1.1,1.1,0.0,0.05,0.00,0.05,0.05,0.05,24.3,11,6,0,54.5,0.45,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,38,28,9,1,99,29,70,93,2.45,78.3,41.1,37.2,0.98,66603.000,1,0,0,201718#
4,882,Manu Garcí­a,es ESP,MF,Alavés,31,1986,1800000,183,midfield - Defensive Midfield,left,La Liga,30,27,2438,3,0,0,2,9,0,0.11,0.00,0.11,0.11,0.11,3.1,1.6,1.4,0.11,0.05,0.17,0.06,0.11,27.1,26,8,0,30.8,0.96,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14,38,15,2,21,40,50,-10,47,1.24,39.0,53.2,-14.2,-0.37,16.819,0,0,0,201718#
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2227,1864,Victor Osimhen,ng NGA,"FW,MF",Wolfsburg,18,1998,1000000,186,attack - Centre-Forward,right,Bundesliga,12,3,287,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,1.1,1.1,0.3,0.37,0.08,0.45,0.37,0.45,3.2,8,2,0,25.0,2.51,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16,34,6,15,13,36

In [8]:
#Integer Encoding of player_positions: Training Set 1
#Will asociate similar positions. Ex: RB and LB
for i in range(len(datos_train1.index)): #Taking best position only
  datos_train1['player_positions'][i] = datos_train1.player_positions[i].split(',',1)[0]

for i in range (len(datos_train1.index)): 
  if datos_train1['player_positions'][i] == 'GK':
    datos_train1['player_positions'][i] = 1
  if datos_train1['player_positions'][i] == 'LB' or datos_train1['player_positions'][i] == 'RB' or datos_train1['player_positions'][i] == 'LWB' or datos_train1['player_positions'][i] == 'RWB':
    datos_train1['player_positions'][i] = 2
  if datos_train1['player_positions'][i] == 'CB':
    datos_train1['player_positions'][i] = 3
  if datos_train1['player_positions'][i] == 'RM' or datos_train1['player_positions'][i] == 'LM':
    datos_train1['player_positions'][i] = 4
  if datos_train1['player_positions'][i] == 'CDM':
    datos_train1['player_positions'][i] = 5
  if datos_train1['player_positions'][i] == 'CM':
    datos_train1['player_positions'][i] = 6
  if datos_train1['player_positions'][i] == 'CAM':
    datos_train1['player_positions'][i] = 7        
  if datos_train1['player_positions'][i] == 'CF':
    datos_train1['player_positions'][i] = 8
  if datos_train1['player_positions'][i] == 'LW' or datos_train1['player_positions'][i] == 'RW':
    datos_train1['player_positions'][i] = 9 
  if datos_train1['player_positions'][i] == 'ST':
    datos_train1['player_positions'][i] = 10       

datos_train1

,sofifa_id,long_name,age,height_cm,weight_kg,overall,potential,player_positions,preferred_foot,international_reputation
0,20801,Cristiano Ronaldo dos Santos Aveiro,32,185,80,94,94,9,Right,5
1,158023,Lionel Andrés Messi Cuccittini,30,170,72,93,93,9,Left,5
2,190871,Neymar da Silva Santos Junior,25,175,68,92,94,9,Right,5
3,167495,Manuel Neuer,31,193,92,92,92,1,Right,5
4,176580,Luis Alberto Suárez Díaz,30,182,86,92,92,10,Right,5
...,...,...,...,...,...,...,...,...,...,...
17587,237463,Adam Kelsey,17,188,74,46,63,1,Right,1
17588,231381,Jordan Young,17,175,71,46,61,10,Left,1
17589,240404,Jack Keeble,18,172,66,46,56,3,Right,1
17590,11728,Barry Richardson,47,185,77,46,46,1,Right,1


In [9]:
#Integer Encoding of player_positions: Training Set 2
#Will asociate similar positions. Ex: RB and LB
for i in range(len(datos_train3.index)): #Taking best position only
  datos_train3['player_positions'][i] = datos_train3.player_positions[i].split(',',1)[0]

for i in range (len(datos_train3.index)): 
  if datos_train3['player_positions'][i] == 'GK':
    datos_train3['player_positions'][i] = 1
  if datos_train3['player_positions'][i] == 'LB' or datos_train3['player_positions'][i] == 'RB' or datos_train3['player_positions'][i] == 'LWB' or datos_train3['player_positions'][i] == 'RWB':
    datos_train3['player_positions'][i] = 2
  if datos_train3['player_positions'][i] == 'CB':
    datos_train3['player_positions'][i] = 3
  if datos_train3['player_positions'][i] == 'RM' or datos_train3['player_positions'][i] == 'LM':
    datos_train3['player_positions'][i] = 4
  if datos_train3['player_positions'][i] == 'CDM':
    datos_train3['player_positions'][i] = 5
  if datos_train3['player_positions'][i] == 'CM':
    datos_train3['player_positions'][i] = 6
  if datos_train3['player_positions'][i] == 'CAM':
    datos_train3['player_positions'][i] = 7        
  if datos_train3['player_positions'][i] == 'CF':
    datos_train3['player_positions'][i] = 8
  if datos_train3['player_positions'][i] == 'LW' or datos_train3['player_positions'][i] == 'RW':
    datos_train3['player_positions'][i] = 9 
  if datos_train3['player_positions'][i] == 'ST':
    datos_train3['player_positions'][i] = 10       

datos_train3

,sofifa_id,long_name,age,height_cm,weight_kg,overall,potential,player_positions,preferred_foot,international_reputation
0,20801,Cristiano Ronaldo dos Santos Aveiro,33,187,83,94,94,10,Right,5
1,158023,Lionel Andrés Messi Cuccittini,31,170,72,94,94,8,Left,5
2,190871,Neymar da Silva Santos Junior,26,175,68,92,93,9,Right,5
3,193080,David De Gea Quintana,27,193,76,91,93,1,Right,4
4,192985,Kevin De Bruyne,27,181,70,91,92,7,Right,4
...,...,...,...,...,...,...,...,...,...,...
17765,238985,Pierce Phillips,19,180,72,47,61,6,Right,1
17766,240160,Alexander Kaltner,18,178,75,47,61,10,Right,1
17767,241304,马俊亮,20,181,64,47,61,6,Right,1
17768,240158,Christoph Ehlich,19,179,73,47,59,2,Right,1


In [10]:
#Integer Encoding of player_positions: Test Set
#Will asociate similar positions. Ex: RB and LB
for i in range(len(datos_test.index)): #Taking best position only
  datos_test['player_positions'][i] = datos_test.player_positions[i].split(',',1)[0]

for i in range (len(datos_test.index)): 
  if datos_test['player_positions'][i] == 'GK':
    datos_test['player_positions'][i] = 1
  if datos_test['player_positions'][i] == 'LB' or datos_test['player_positions'][i] == 'RB' or datos_test['player_positions'][i] == 'LWB' or datos_test['player_positions'][i] == 'RWB':
    datos_test['player_positions'][i] = 2
  if datos_test['player_positions'][i] == 'CB':
    datos_test['player_positions'][i] = 3
  if datos_test['player_positions'][i] == 'RM' or datos_test['player_positions'][i] == 'LM':
    datos_test['player_positions'][i] = 4
  if datos_test['player_positions'][i] == 'CDM':
    datos_test['player_positions'][i] = 5
  if datos_test['player_positions'][i] == 'CM':
    datos_test['player_positions'][i] = 6
  if datos_test['player_positions'][i] == 'CAM':
    datos_test['player_positions'][i] = 7        
  if datos_test['player_positions'][i] == 'CF':
    datos_test['player_positions'][i] = 8
  if datos_test['player_positions'][i] == 'LW' or datos_test['player_positions'][i] == 'RW':
    datos_test['player_positions'][i] = 9 
  if datos_test['player_positions'][i] == 'ST':
    datos_test['player_positions'][i] = 10       

datos_test

,sofifa_id,long_name,age,height_cm,weight_kg,overall,potential,player_positions,preferred_foot,international_reputation
0,158023,Lionel Andrés Messi Cuccittini,32,170,72,94,94,9,Left,5
1,20801,Cristiano Ronaldo dos Santos Aveiro,34,187,83,93,93,10,Right,5
2,190871,Neymar da Silva Santos Junior,27,175,68,92,92,9,Right,5
3,200389,Jan Oblak,26,188,87,91,93,1,Right,3
4,183277,Eden Hazard,28,175,74,91,91,9,Right,4
...,...,...,...,...,...,...,...,...,...,...
18273,245006,邵帅,22,186,79,48,56,3,Right,1
18274,250995,Mingjie Xiao,22,177,66,48,56,3,Right,1
18275,252332,张威,19,186,75,48,56,6,Right,1
18276,251110,汪海健,18,185,74,48,54,6,Right,1


In [11]:
#Normalizing values of overall and potential
#Training Set 1,2 and Test Set

datos_train1['overall'] = datos_train1['overall'].div(100)
datos_train3['overall'] = datos_train3['overall'].div(100)
datos_test['overall'] = datos_test['overall'].div(100)
datos_train1['potential'] = datos_train1['potential'].div(100)
datos_train3['potential'] = datos_train3['potential'].div(100)
datos_test['potential'] = datos_test['potential'].div(100)

In [12]:
#One Hot Encoding of preferred_foot: Training Set 1

datos_train1['right_foot'] = np.zeros
datos_train1['left_foot'] = np.zeros

for i in range (len(datos_train1.index)): 
  if datos_train1['preferred_foot'][i] == 'Left':
    datos_train1['left_foot'][i] = 1
    datos_train1['right_foot'][i] = 0
  if datos_train1['preferred_foot'][i] == 'Right':
    datos_train1['right_foot'][i] = 1
    datos_train1['left_foot'][i] = 0

datos_train1 = datos_train1.drop('preferred_foot', axis = 1)
datos_train1    

,sofifa_id,long_name,age,height_cm,weight_kg,overall,potential,player_positions,international_reputation,right_foot,left_foot
0,20801,Cristiano Ronaldo dos Santos Aveiro,32,185,80,0.94,0.94,9,5,1,0
1,158023,Lionel Andrés Messi Cuccittini,30,170,72,0.93,0.93,9,5,0,1
2,190871,Neymar da Silva Santos Junior,25,175,68,0.92,0.94,9,5,1,0
3,167495,Manuel Neuer,31,193,92,0.92,0.92,1,5,1,0
4,176580,Luis Alberto Suárez Díaz,30,182,86,0.92,0.92,10,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...
17587,237463,Adam Kelsey,17,188,74,0.46,0.63,1,1,1,0
17588,231381,Jordan Young,17,175,71,0.46,0.61,10,1,0,1
17589,240404,Jack Keeble,18,172,66,0.46,0.56,3,1,1,0
17590,11728,Barry Richardson,47,185,77,0.46,0.46,1,1,1,0


In [13]:
#One Hot Encoding of preferred_foot: Training Set 2

datos_train3['right_foot'] = np.zeros
datos_train3['left_foot'] = np.zeros

for i in range (len(datos_train3.index)): 
  if datos_train3['preferred_foot'][i] == 'Left':
    datos_train3['left_foot'][i] = 1
    datos_train3['right_foot'][i] = 0
  if datos_train3['preferred_foot'][i] == 'Right':
    datos_train3['right_foot'][i] = 1
    datos_train3['left_foot'][i] = 0

datos_train3 = datos_train3.drop('preferred_foot', axis = 1)
datos_train3   

,sofifa_id,long_name,age,height_cm,weight_kg,overall,potential,player_positions,international_reputation,right_foot,left_foot
0,20801,Cristiano Ronaldo dos Santos Aveiro,33,187,83,0.94,0.94,10,5,1,0
1,158023,Lionel Andrés Messi Cuccittini,31,170,72,0.94,0.94,8,5,0,1
2,190871,Neymar da Silva Santos Junior,26,175,68,0.92,0.93,9,5,1,0
3,193080,David De Gea Quintana,27,193,76,0.91,0.93,1,4,1,0
4,192985,Kevin De Bruyne,27,181,70,0.91,0.92,7,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...
17765,238985,Pierce Phillips,19,180,72,0.47,0.61,6,1,1,0
17766,240160,Alexander Kaltner,18,178,75,0.47,0.61,10,1,1,0
17767,241304,马俊亮,20,181,64,0.47,0.61,6,1,1,0
17768,240158,Christoph Ehlich,19,179,73,0.47,0.59,2,1,1,0


In [14]:
#One Hot Encoding of preferred_foot: Test Set

datos_test['right_foot'] = np.zeros
datos_test['left_foot'] = np.zeros

for i in range (len(datos_test.index)): 
  if datos_test['preferred_foot'][i] == 'Left':
    datos_test['left_foot'][i] = 1
    datos_test['right_foot'][i] = 0
  if datos_test['preferred_foot'][i] == 'Right':
    datos_test['right_foot'][i] = 1
    datos_test['left_foot'][i] = 0

datos_test = datos_test.drop('preferred_foot', axis = 1)
datos_test  

,sofifa_id,long_name,age,height_cm,weight_kg,overall,potential,player_positions,international_reputation,right_foot,left_foot
0,158023,Lionel Andrés Messi Cuccittini,32,170,72,0.94,0.94,9,5,0,1
1,20801,Cristiano Ronaldo dos Santos Aveiro,34,187,83,0.93,0.93,10,5,1,0
2,190871,Neymar da Silva Santos Junior,27,175,68,0.92,0.92,9,5,1,0
3,200389,Jan Oblak,26,188,87,0.91,0.93,1,3,1,0
4,183277,Eden Hazard,28,175,74,0.91,0.91,9,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...
18273,245006,邵帅,22,186,79,0.48,0.56,3,1,1,0
18274,250995,Mingjie Xiao,22,177,66,0.48,0.56,3,1,1,0
18275,252332,张威,19,186,75,0.48,0.56,6,1,1,0
18276,251110,汪海健,18,185,74,0.48,0.54,6,1,1,0


In [15]:
#League Division by clubs
#If player is in 1st division, league_division is 1, 2nd division is 2 and so on
#Not implemented yet

In [16]:
#Preprocessing Transfermarkt Data

In [17]:
#Values to replace in excel using search and replace function (transfermarkt database 1718 and 1819):
#Ã© -- é
#Ã³ -- ó
#Ã¡ -- á
#Ã« -- ë
#Ãº -- ú
#Ã¯ -- ï
#Ã´ -- ô
#Ã¼ -- ü
#Ã– -- Ö
#Ã -- í
#í± -- ñ
#í¶ -- ö
#í² -- ò
#í‰ -- É
#í“ -- Ó
#È™ -- ș
#í® -- î
#Ä‡ -- ć
#í£ -- ã
#í‘ -- Ñ
#í§ -- ç

In [18]:
#Values to replace in excel using search and replace function (transfermarkt database 1920):
#Ăş -- ú
#Ă© -- é
#ĂŻ -- ï
#Ã -- í
#ÄŤ -- č
#í§ -- ç
#Ĺˇ -- š
#íˇ -- á
#í˛ -- ò
#í« -- ë
#í‰ -- É
#í¶ -- ö
#í– -- Ö
#í§ -- ç
#í± -- ñ
#íł -- ó
#í‘ -- Ñ
#í“ -- Ó
#í´ -- ô
#Ä‡ -- ć
#Ã¼ -- ü
#í“ -- Ó

In [19]:
#Deleting Non-useful columns
#no_use_columns2 = ['Unnamed: 0','nationality','position','squad','age',
#                  'birth_year','height','position2','foot',
#                  'league','CLBestScorer','Season','passes_total_distance', 'passes_progressive_distance', 
#                   'passes_total_distancem','passes_progressive_distancem', 'carry_distance', 'carry_progressive_distance',
#                   'carry_distancem','carry_progressive_distancem','MP','Attendance','W', 'D', 'L', 'GF', 'GA','GDiff',
#                   'Pts', 	'Pts/G', 	'xG', 	'xGA', 	'xGDiff', 	'xGDiff/90', 'cards_yellow', 'cards_red','passes_completed',
#                   'passes','passes_total_distance','passes_progressive_distance','passes_completed_short','passes_short','passes_completed_medium',
#                  'passes_medium','passes_live', 'passes_dead', 'passes_free_kicks','through_balls','passes_pressure','passes_switches',
#                   'corner_kicks','corner_kicks_in','corner_kicks_out', 'corner_kicks_straight','passes_ground','passes_low','passes_high',
#                   'passes_left_foot', 'passes_right_foot']

useful_columns = ['player','games','games_starts','minutes','goals','assists','goals_per90','assists_per90','goals_assists_per90','goals_pens_per90',
               'goals_assists_pens_per90','xg','xa','xg_per90','xa_per90','shots_on_target_pct',
               'shots_total_per90','shots_on_target_per90','goals_per_shot','goals_per_shot_on_target',
               'passes_pct','crosses','passes_intercepted','passes_blocked','tackles','tackles_won','dribble_tackles_pct','dribbled_past',
               'pressure_regain_pct','blocks','blocked_shots','blocked_passes','interceptions','clearances','errors','touches_def_pen_area',
               'touches_att_pen_area','dribbles_completed_pct','players_dribbled_past','passes_received_pct',
               'miscontrols','dispossessed','fouls','fouled','offsides','pens_won','pens_conceded','ball_recoveries','aerials_won_pct',
               'games_gk','games_starts_gk','minutes_gk','goals_against_gk','shots_on_target_against','saves','save_pct','clean_sheets',
               'pens_saved','LgRk','CL']

datos_train2 = datos_train2[useful_columns]
datos_test2 = datos_test2[useful_columns]
datos_train4 = datos_train4[useful_columns]
datos_train2

,player,games,games_starts,minutes,goals,assists,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,xa,xg_per90,xa_per90,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,passes_pct,crosses,passes_intercepted,passes_blocked,tackles,tackles_won,dribble_tackles_pct,dribbled_past,pressure_regain_pct,blocks,blocked_shots,blocked_passes,interceptions,clearances,errors,touches_def_pen_area,touches_att_pen_area,dribbles_completed_pct,players_dribbled_past,passes_received_pct,miscontrols,dispossessed,fouls,fouled,offsides,pens_won,pens_conceded,ball_recoveries,aerials_won_pct,games_gk,games_starts_gk,minutes_gk,goals_against_gk,shots_on_target_against,saves,save_pct,clean_sheets,pens_saved,LgRk,CL
0,Burgui,23,12,1037,1,1,0.09,0.09,0.17,0.09,0.17,0.9,1.7,0.08,0.15,28.6,1.22,0.35,0.07,0.25,74.6,18,14,22,12,7,42.9,4,23.6,9,0,9,4,1,0,6,29,77.4,25,73.7,24,20,10,19,5,1,0,56,8.3,0,0,0,0,0,0,0.0,0,0,14,0
1,Raphaël Varane,27,27,2320,0,1,0.00,0.04,0.04,0.00,0.04,0.9,0.9,0.04,0.04,20.0,0.39,0.08,0.00,0.00,88.4,1,13,12,22,14,33.3,10,35.5,32,14,18,17,80,2,179,19,83.3,10,98.1,3,3,17,12,0,0,0,272,73.8,0,0,0,0,0,0,0.0,0,0,3,1
2,Rubén Duarte,24,24,2138,0,2,0.00,0.08,0.08,0.00,0.08,0.5,1.6,0.02,0.07,12.5,0.34,0.04,0.00,0.00,68.5,36,12,25,54,29,36.7,31,24.1,74,11,63,30,51,0,107,19,91.7,11,90.6,5,13,48,35,1,0,2,283,59.5,0,0,0,0,0,0,0.0,0,0,14,0
3,Samuel Umtiti,25,24,2189,1,0,0.04,0.00,0.04,0.04,0.04,1.1,0.0,0.05,0.00,54.5,0.45,0.25,0.09,0.17,90.2,0,10,23,40,26,52.2,11,40.3,30,15,15,30,55,0,214,17,75.0,6,98.3,7,4,23,25,4,0,0,248,77.4,0,0,0,0,0,0,0.0,0,0,1,1
4,Manu Garcí­a,30,27,2438,3,0,0.11,0.00,0.11,0.11,0.11,3.1,1.4,0.11,0.05,30.8,0.96,0.30,0.12,0.38,72.6,11,19,27,90,69,25.6,58,23.6,40,9,31,32,55,0,82,39,81.0,17,84.6,23,22,66,72,5,0,0,292,58.3,0,0,0,0,0,0,0.0,0,0,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2227,Victor Osimhen,12,3,287,0,0,0.00,0.00,0.00,0.00,0.00,1.1,0.3,0.37,0.08,25.0,2.51,0.63,0.00,0.00,67.8,4,3,6,2,1,0.0,2,31.6,1,1,0,2,7,0,7,22,12.5,2,59.0,18,7,9,8,2,0,0,22,40.0,0,0,0,0,0,0,0.0,0,0,16,0
2228,Renato Steffen,16,6,659,0,0,0.00,0.00,0.00,0.00,0.00,1.3,1.1,0.18,0.16,20.0,0.68,0.14,0.00,0.00,77.0,11,3,7,10,6,44.4,5,19.9,8,0,8,1,10,0,12,12,54.5,6,76.7,13,6,20,15,0,0,1,45,5.0,0,0,0,0,0,0,0.0,0,0,16,0
2229,Paul Jaeckel,3,3,270,0,0,0.00,0.00,0.00,0.00,0.00,0.2,0.2,0.08,0.07,0.0,1.00,0.00,0.00,0.00,81.0,4,0,2,4,4,25.0,3,26.7,4,2,2,8,17,0,13,3,100.0,2,100.0,0,2,3,0,0,0,0,22,76.9,0,0,0,0,0,0,0.0,0,0,16,0
2230,Paul Verhaegh,31,30,2614,2,2,0.07,0.07,0.14,0.03,0.10,2.6,1.5,0.09,0.05,66.7,0.10,0.07,0.33,0.50,78.0,43,32,51,30,14,42.9,16,28.9,58,12,46,21,74,0,100,13,68.2,16,93.7,19,17,20,33,0,0,0,252,59.1,0,0,0,0,0,0,0.0,0,0,16,0


In [20]:
#Applying normalization to league ranking (LgRk)

datos_train2['LgRk'] = (datos_train2['LgRk'] - datos_train2['LgRk'].min()) / (datos_train2['LgRk'].max() - datos_train2['LgRk'].min())
datos_train4['LgRk'] = (datos_train4['LgRk'] - datos_train4['LgRk'].min()) / (datos_train4['LgRk'].max() - datos_train4['LgRk'].min())
datos_test2['LgRk'] = (datos_test2['LgRk'] - datos_test2['LgRk'].min()) / (datos_test2['LgRk'].max() - datos_test2['LgRk'].min())

#Applying normalization to minutes played (minutes)

datos_train2['minutes'] = (datos_train2['minutes'] - datos_train2['minutes'].min()) / (datos_train2['minutes'].max() - datos_train2['minutes'].min())
datos_train4['minutes'] = (datos_train4['minutes'] - datos_train4['minutes'].min()) / (datos_train4['minutes'].max() - datos_train4['minutes'].min())
datos_test2['minutes'] = (datos_test2['minutes'] - datos_test2['minutes'].min()) / (datos_test2['minutes'].max() - datos_test2['minutes'].min())

#Dividing percentages (100)

datos_train2['shots_on_target_pct'] = datos_train2['shots_on_target_pct'].div(100)
datos_train4['shots_on_target_pct'] = datos_train2['shots_on_target_pct'].div(100)
datos_test2['shots_on_target_pct'] = datos_test2['shots_on_target_pct'].div(100)

datos_train2['passes_pct'] = datos_train2['passes_pct'].div(100)
datos_train4['passes_pct'] = datos_train2['passes_pct'].div(100)
datos_test2['passes_pct'] = datos_test2['passes_pct'].div(100)

datos_train2['dribbles_completed_pct'] = datos_train2['dribbles_completed_pct'].div(100)
datos_train4['dribbles_completed_pct'] = datos_train2['dribbles_completed_pct'].div(100)
datos_test2['dribbles_completed_pct'] = datos_test2['dribbles_completed_pct'].div(100)

datos_train2['dribble_tackles_pct'] = datos_train2['dribble_tackles_pct'].div(100)
datos_train4['dribble_tackles_pct'] = datos_train2['dribble_tackles_pct'].div(100)
datos_test2['dribble_tackles_pct'] = datos_test2['dribble_tackles_pct'].div(100)

datos_train2['pressure_regain_pct'] = datos_train2['pressure_regain_pct'].div(100)
datos_train4['pressure_regain_pct'] = datos_train2['pressure_regain_pct'].div(100)
datos_test2['pressure_regain_pct'] = datos_test2['pressure_regain_pct'].div(100)

datos_train2['dribbles_completed_pct'] = datos_train2['dribbles_completed_pct'].div(100)
datos_train4['dribbles_completed_pct'] = datos_train2['dribbles_completed_pct'].div(100)
datos_test2['dribbles_completed_pct'] = datos_test2['dribbles_completed_pct'].div(100)

datos_train2['passes_received_pct'] = datos_train2['passes_received_pct'].div(100)
datos_train4['passes_received_pct'] = datos_train2['passes_received_pct'].div(100)
datos_test2['passes_received_pct'] = datos_test2['passes_received_pct'].div(100)

datos_train2['aerials_won_pct'] = datos_train2['aerials_won_pct'].div(100)
datos_train4['aerials_won_pct'] = datos_train2['aerials_won_pct'].div(100)
datos_test2['aerials_won_pct'] = datos_test2['aerials_won_pct'].div(100)

datos_train2['save_pct'] = datos_train2['save_pct'].div(100)
datos_train4['save_pct'] = datos_train2['save_pct'].div(100)
datos_test2['save_pct'] = datos_test2['save_pct'].div(100)
datos_train2

,player,games,games_starts,minutes,goals,assists,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,xa,xg_per90,xa_per90,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,passes_pct,crosses,passes_intercepted,passes_blocked,tackles,tackles_won,dribble_tackles_pct,dribbled_past,pressure_regain_pct,blocks,blocked_shots,blocked_passes,interceptions,clearances,errors,touches_def_pen_area,touches_att_pen_area,dribbles_completed_pct,players_dribbled_past,passes_received_pct,miscontrols,dispossessed,fouls,fouled,offsides,pens_won,pens_conceded,ball_recoveries,aerials_won_pct,games_gk,games_starts_gk,minutes_gk,goals_against_gk,shots_on_target_against,saves,save_pct,clean_sheets,pens_saved,LgRk,CL
0,Burgui,23,12,0.303013,1,1,0.09,0.09,0.17,0.09,0.17,0.9,1.7,0.08,0.15,0.286,1.22,0.35,0.07,0.25,0.746,18,14,22,12,7,0.429,4,0.236,9,0,9,4,1,0,6,29,0.00774,25,0.737,24,20,10,19,5,1,0,56,0.083,0,0,0,0,0,0,0.0,0,0,0.684211,0
1,Raphaël Varane,27,27,0.678268,0,1,0.00,0.04,0.04,0.00,0.04,0.9,0.9,0.04,0.04,0.200,0.39,0.08,0.00,0.00,0.884,1,13,12,22,14,0.333,10,0.355,32,14,18,17,80,2,179,19,0.00833,10,0.981,3,3,17,12,0,0,0,272,0.738,0,0,0,0,0,0,0.0,0,0,0.105263,1
2,Rubén Duarte,24,24,0.625037,0,2,0.00,0.08,0.08,0.00,0.08,0.5,1.6,0.02,0.07,0.125,0.34,0.04,0.00,0.00,0.685,36,12,25,54,29,0.367,31,0.241,74,11,63,30,51,0,107,19,0.00917,11,0.906,5,13,48,35,1,0,2,283,0.595,0,0,0,0,0,0,0.0,0,0,0.684211,0
3,Samuel Umtiti,25,24,0.639953,1,0,0.04,0.00,0.04,0.04,0.04,1.1,0.0,0.05,0.00,0.545,0.45,0.25,0.09,0.17,0.902,0,10,23,40,26,0.522,11,0.403,30,15,15,30,55,0,214,17,0.00750,6,0.983,7,4,23,25,4,0,0,248,0.774,0,0,0,0,0,0,0.0,0,0,0.000000,1
4,Manu Garcí­a,30,27,0.712782,3,0,0.11,0.00,0.11,0.11,0.11,3.1,1.4,0.11,0.05,0.308,0.96,0.30,0.12,0.38,0.726,11,19,27,90,69,0.256,58,0.236,40,9,31,32,55,0,82,39,0.00810,17,0.846,23,22,66,72,5,0,0,292,0.583,0,0,0,0,0,0,0.0,0,0,0.684211,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2227,Victor Osimhen,12,3,0.083650,0,0,0.00,0.00,0.00,0.00,0.00,1.1,0.3,0.37,0.08,0.250,2.51,0.63,0.00,0.00,0.678,4,3,6,2,1,0.000,2,0.316,1,1,0,2,7,0,7,22,0.00125,2,0.590,18,7,9,8,2,0,0,22,0.400,0,0,0,0,0,0,0.0,0,0,0.789474,0
2228,Renato Steffen,16,6,0.192454,0,0,0.00,0.00,0.00,0.00,0.00,1.3,1.1,0.18,0.16,0.200,0.68,0.14,0.00,0.00,0.770,11,3,7,10,6,0.444,5,0.199,8,0,8,1,10,0,12,12,0.00545,6,0.767,13,6,20,15,0,0,1,45,0.050,0,0,0,0,0,0,0.0,0,0,0.789474,0
2229,Paul Jaeckel,3,3,0.078678,0,0,0.00,0.00,0.00,0.00,0.00,0.2,0.2,0.08,0.07,0.000,1.00,0.00,0.00,0.00,0.810,4,0,2,4,4,0.250,3,0.267,4,2,2,8,17,0,13,3,0.01000,2,1.000,0,2,3,0,0,0,0,22,0.769,0,0,0,0,0,0,0.0,0,0,0.789474,0
2230,Paul Verhaegh,31,30,0.764259,2,2,0.07,0.07,0.14,0.03,0.10,2.6,1.5,0.09,0.05,0.667,0.10,0.07,0.33,0.50,0.780,43,32,51,30,14,0.429,16,0.289,58,12,46,21,74,0,100,13,0.00682,16,0.937,19,17,20,33,0,0,0,252,0.591,0,0,0,0,0,0,0.0,0,0,0.789474,0


In [21]:
#e = datos_train2.loc[[198]]
#e
d = np.where(datos_train2['player'] == 'Neymar')
d
#d = np.where(datos_train1['long_name'] == 'Victor Osimhen')
#d
#c = datos_train1.loc[[57]]
#c
#fullstring = "Lionel Messi Cucittini"
#substring = "Lionel Messi"
#if substring in fullstring:
#    print("Found!")
#else:
#    print("Not found!")

(array([1580]),)

In [22]:
#Merging of sofifa and transfermarkt databases by player name: Training Set 1
#Comparing player names to set as key

datos_train_list1 = datos_train1.values.tolist()
datos_train_list2 = datos_train2.values.tolist()

for i in range (len(datos_train_list2)):
 for j in range (len(datos_train_list1)):
  if datos_train_list2[i][0] in datos_train_list1[j][1]:        
          datos_train_list1[j][1] = datos_train_list2[i][0]

datos_train_list1[2][1]

'Neymar'

In [23]:
#Merging of sofifa and transfermarkt databases by player name: Training Set 2
#Comparing player names to set as key

datos_train_list3 = datos_train3.values.tolist()
datos_train_list4 = datos_train4.values.tolist()

for i in range (len(datos_train_list4)):
 for j in range (len(datos_train_list3)):
  if str(datos_train_list4[i][0]) in str(datos_train_list3[j][1]):        
          datos_train_list3[j][1] = datos_train_list4[i][0]

datos_train_list3[2][1]

'Neymar'

In [24]:
#Merging of sofifa and transfermarkt databases by player name: Test Set
#Comparing player names to set as key

datos_test_list1 = datos_test.values.tolist()
datos_test_list2 = datos_test2.values.tolist()

for i in range (len(datos_test_list2)):
  for j in range (len(datos_test_list1)):
     if str(datos_test_list2[i][0]) in str(datos_test_list1[j][1]):        
          datos_test_list1[j][1] = datos_test_list2[i][0]

datos_test_list1[2][1]

'Neymar'

In [25]:
#Returning lists to dataframes
datos_train1 = pd.DataFrame(datos_train_list1, columns = ['sofifa_id', 	'long_name', 	'age', 	'height_cm', 	'weight_kg', 	'overall', 	'potential', 	
                                        'player_positions', 	'international_reputation', 'right_foot', 'left_foot'])

datos_train3 = pd.DataFrame(datos_train_list3, columns = ['sofifa_id', 	'long_name', 	'age', 	'height_cm', 	'weight_kg', 	'overall', 	'potential', 	
                                        'player_positions', 	'international_reputation', 'right_foot', 'left_foot'])

datos_test = pd.DataFrame(datos_test_list1, columns = ['sofifa_id', 	'long_name', 	'age', 	'height_cm', 	'weight_kg', 	'overall', 	'potential', 	
                                        'player_positions', 	'international_reputation', 'right_foot', 'left_foot'])


datos_train2.rename(columns={'player': 'long_name'}, inplace=True)
datos_test2.rename(columns={'player': 'long_name'}, inplace=True)
datos_train4.rename(columns={'player': 'long_name'}, inplace=True)
datos_test_nuevo.rename(columns={'ID': 'sofifa_id'}, inplace=True)
datos_train2

,long_name,games,games_starts,minutes,goals,assists,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,xa,xg_per90,xa_per90,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,passes_pct,crosses,passes_intercepted,passes_blocked,tackles,tackles_won,dribble_tackles_pct,dribbled_past,pressure_regain_pct,blocks,blocked_shots,blocked_passes,interceptions,clearances,errors,touches_def_pen_area,touches_att_pen_area,dribbles_completed_pct,players_dribbled_past,passes_received_pct,miscontrols,dispossessed,fouls,fouled,offsides,pens_won,pens_conceded,ball_recoveries,aerials_won_pct,games_gk,games_starts_gk,minutes_gk,goals_against_gk,shots_on_target_against,saves,save_pct,clean_sheets,pens_saved,LgRk,CL
0,Burgui,23,12,0.303013,1,1,0.09,0.09,0.17,0.09,0.17,0.9,1.7,0.08,0.15,0.286,1.22,0.35,0.07,0.25,0.746,18,14,22,12,7,0.429,4,0.236,9,0,9,4,1,0,6,29,0.00774,25,0.737,24,20,10,19,5,1,0,56,0.083,0,0,0,0,0,0,0.0,0,0,0.684211,0
1,Raphaël Varane,27,27,0.678268,0,1,0.00,0.04,0.04,0.00,0.04,0.9,0.9,0.04,0.04,0.200,0.39,0.08,0.00,0.00,0.884,1,13,12,22,14,0.333,10,0.355,32,14,18,17,80,2,179,19,0.00833,10,0.981,3,3,17,12,0,0,0,272,0.738,0,0,0,0,0,0,0.0,0,0,0.105263,1
2,Rubén Duarte,24,24,0.625037,0,2,0.00,0.08,0.08,0.00,0.08,0.5,1.6,0.02,0.07,0.125,0.34,0.04,0.00,0.00,0.685,36,12,25,54,29,0.367,31,0.241,74,11,63,30,51,0,107,19,0.00917,11,0.906,5,13,48,35,1,0,2,283,0.595,0,0,0,0,0,0,0.0,0,0,0.684211,0
3,Samuel Umtiti,25,24,0.639953,1,0,0.04,0.00,0.04,0.04,0.04,1.1,0.0,0.05,0.00,0.545,0.45,0.25,0.09,0.17,0.902,0,10,23,40,26,0.522,11,0.403,30,15,15,30,55,0,214,17,0.00750,6,0.983,7,4,23,25,4,0,0,248,0.774,0,0,0,0,0,0,0.0,0,0,0.000000,1
4,Manu Garcí­a,30,27,0.712782,3,0,0.11,0.00,0.11,0.11,0.11,3.1,1.4,0.11,0.05,0.308,0.96,0.30,0.12,0.38,0.726,11,19,27,90,69,0.256,58,0.236,40,9,31,32,55,0,82,39,0.00810,17,0.846,23,22,66,72,5,0,0,292,0.583,0,0,0,0,0,0,0.0,0,0,0.684211,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2227,Victor Osimhen,12,3,0.083650,0,0,0.00,0.00,0.00,0.00,0.00,1.1,0.3,0.37,0.08,0.250,2.51,0.63,0.00,0.00,0.678,4,3,6,2,1,0.000,2,0.316,1,1,0,2,7,0,7,22,0.00125,2,0.590,18,7,9,8,2,0,0,22,0.400,0,0,0,0,0,0,0.0,0,0,0.789474,0
2228,Renato Steffen,16,6,0.192454,0,0,0.00,0.00,0.00,0.00,0.00,1.3,1.1,0.18,0.16,0.200,0.68,0.14,0.00,0.00,0.770,11,3,7,10,6,0.444,5,0.199,8,0,8,1,10,0,12,12,0.00545,6,0.767,13,6,20,15,0,0,1,45,0.050,0,0,0,0,0,0,0.0,0,0,0.789474,0
2229,Paul Jaeckel,3,3,0.078678,0,0,0.00,0.00,0.00,0.00,0.00,0.2,0.2,0.08,0.07,0.000,1.00,0.00,0.00,0.00,0.810,4,0,2,4,4,0.250,3,0.267,4,2,2,8,17,0,13,3,0.01000,2,1.000,0,2,3,0,0,0,0,22,0.769,0,0,0,0,0,0,0.0,0,0,0.789474,0
2230,Paul Verhaegh,31,30,0.764259,2,2,0.07,0.07,0.14,0.03,0.10,2.6,1.5,0.09,0.05,0.667,0.10,0.07,0.33,0.50,0.780,43,32,51,30,14,0.429,16,0.289,58,12,46,21,74,0,100,13,0.00682,16,0.937,19,17,20,33,0,0,0,252,0.591,0,0,0,0,0,0,0.0,0,0,0.789474,0


In [26]:
datos_test2

,long_name,games,games_starts,minutes,goals,assists,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,xa,xg_per90,xa_per90,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,passes_pct,crosses,passes_intercepted,passes_blocked,tackles,tackles_won,dribble_tackles_pct,dribbled_past,pressure_regain_pct,blocks,blocked_shots,blocked_passes,interceptions,clearances,errors,touches_def_pen_area,touches_att_pen_area,dribbles_completed_pct,players_dribbled_past,passes_received_pct,miscontrols,dispossessed,fouls,fouled,offsides,pens_won,pens_conceded,ball_recoveries,aerials_won_pct,games_gk,games_starts_gk,minutes_gk,goals_against_gk,shots_on_target_against,saves,save_pct,clean_sheets,pens_saved,LgRk,CL
0,Martin Aguirregabiria,31,23,0.613630,1,1,0.04,0.04,0.09,0.04,0.09,0.9,1.0,0.04,0.04,0.333,0.13,0.04,0.33,1.00,0.712,30,17,44,50,22,0.500,25,0.242,39,8,31,9,68,1,83,9,0.00538,16,0.898,20,17,32,19,1,0,1,207,0.535,0,0,0,0,0,0,0.00000,0,0,0.789474,0
1,Oliver Burke,31,14,0.417666,1,2,0.06,0.13,0.19,0.06,0.19,3.2,1.2,0.20,0.07,0.450,1.26,0.57,0.05,0.11,0.679,19,5,17,20,5,0.391,14,0.207,25,2,23,6,14,0,23,57,0.00591,29,0.579,32,33,19,11,1,0,0,75,0.377,0,0,0,0,0,0,0.00000,0,0,0.789474,0
2,Ví­ctor Camarasa,17,15,0.377888,0,1,0.00,0.07,0.07,0.00,0.07,0.3,1.6,0.02,0.11,0.250,0.56,0.14,0.00,0.00,0.807,13,8,11,17,14,0.316,13,0.256,16,3,13,10,18,0,24,11,0.00552,16,0.844,23,12,14,15,2,0,0,114,0.444,0,0,0,0,0,0,0.00000,0,0,0.789474,0
3,Rubén Duarte,31,30,0.778590,0,1,0.00,0.03,0.03,0.00,0.03,1.3,0.9,0.04,0.03,0.400,0.68,0.27,0.00,0.00,0.712,44,18,38,69,38,0.545,25,0.299,62,9,53,19,128,1,97,33,0.00400,5,0.923,14,16,45,39,0,1,5,289,0.542,0,0,0,0,0,0,0.00000,0,0,0.789474,0
4,Rodrigo Ely,25,24,0.620064,2,0,0.08,0.00,0.08,0.08,0.08,0.7,0.1,0.03,0.00,0.500,0.25,0.13,0.33,0.67,0.795,0,4,8,27,16,0.370,17,0.306,38,20,18,12,178,1,159,15,0.01000,4,0.956,3,8,34,12,2,0,0,268,0.704,0,0,0,0,0,0,0.00000,0,0,0.789474,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2227,Ryan Bertrand,32,31,0.801696,1,1,0.03,0.03,0.07,0.03,0.07,1.7,3.6,0.06,0.12,0.211,0.62,0.13,0.05,0.25,0.751,81,29,61,48,28,0.345,36,0.369,52,13,39,26,76,1,69,52,0.00308,9,0.883,28,33,27,32,1,0,0,274,0.546,0,0,0,0,0,0,0.00000,0,0,0.526316,0
2228,Sofiane Boufal,20,8,0.223165,0,2,0.00,0.24,0.24,0.00,0.24,1.0,2.7,0.11,0.32,0.333,2.47,0.82,0.00,0.00,0.760,30,6,14,21,12,0.231,10,0.326,17,0,17,5,2,0,6,53,0.00563,61,0.816,21,41,25,20,2,0,1,69,0.421,0,0,0,0,0,0,0.00000,0,0,0.526316,0
2229,Kevin Danso,6,3,0.080725,0,1,0.00,0.32,0.32,0.00,0.32,0.1,0.2,0.04,0.06,0.000,0.32,0.00,0.00,0.00,0.723,6,5,3,8,4,0.625,3,0.244,9,4,5,2,15,0,12,2,0.00500,3,0.886,8,1,4,2,0,0,0,28,0.524,0,0,0,0,0,0,0.00000,0,0,0.526316,0
2230,Moussa Djenepo,18,10,0.271132,2,2,0.19,0.19,0.39,0.19,0.39,2.7,1.3,0.26,0.13,0.350,1.94,0.68,0.10,0.29,0.723,16,8,8,29,18,0.348,15,0.298,24,2,22,7,12,0,15,48,0.00686,49,0.723,34,24,23,30,3,0,0,55,0.263,0,0,0,0,0,0,0.00000,0,0,0.526316,0


In [27]:
# Merging sofifa and transfermarkt databases using merge
datos_train1 = pd.merge(datos_train1, datos_train2 , how = 'inner', on = 'long_name')
datos_train3 = pd.merge(datos_train3, datos_train4 , how = 'inner', on = 'long_name')
datos_test = pd.merge(datos_test, datos_test2 , how = 'inner', on = 'long_name')
datos_train1

,sofifa_id,long_name,age,height_cm,weight_kg,overall,potential,player_positions,international_reputation,right_foot,left_foot,games,games_starts,minutes,goals,assists,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,xa,xg_per90,xa_per90,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,passes_pct,crosses,passes_intercepted,passes_blocked,tackles,tackles_won,dribble_tackles_pct,dribbled_past,pressure_regain_pct,blocks,blocked_shots,blocked_passes,interceptions,clearances,errors,touches_def_pen_area,touches_att_pen_area,dribbles_completed_pct,players_dribbled_past,passes_received_pct,miscontrols,dispossessed,fouls,fouled,offsides,pens_won,pens_conceded,ball_recoveries,aerials_won_pct,games_gk,games_starts_gk,minutes_gk,goals_against_gk,shots_on_target_against,saves,save_pct,clean_sheets,pens_saved,LgRk,CL
0,20801,Cristiano Ronaldo,32,185,80,0.94,0.94,9,5,1,0,27,27,0.668909,26,5,1.02,0.20,1.22,0.90,1.10,24.1,4.3,0.95,0.17,0.403,6.92,2.79,0.13,0.32,0.797,44,19,29,5,3,0.100,9,0.253,14,5,9,6,15,0,19,245,0.00689,32,0.669,47,23,19,18,41,1,0,72,0.595,0,0,0,0,0,0,0.0000,0,0,0.105263,1
1,190871,Neymar,25,175,68,0.92,0.94,9,5,1,0,20,20,0.525300,19,13,0.95,0.65,1.60,0.75,1.40,15.4,11.8,0.78,0.59,0.437,4.36,1.90,0.17,0.39,0.760,58,48,55,15,9,0.259,20,0.285,10,0,10,3,0,0,0,159,0.00624,127,0.886,64,58,24,103,10,2,0,110,0.500,0,0,0,0,0,0,0.0000,0,0,0.000000,1
2,167495,Manuel Neuer,31,193,92,0.92,0.92,1,5,1,0,3,3,0.078678,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.852,0,1,0,0,0,0.000,0,0.000,0,0,0,0,1,0,67,1,0.00000,0,1.000,0,0,0,0,0,0,0,15,0.000,3,3,270,2,8,6,0.0075,2,0,0.000000,1
3,176580,Luis Alberto,30,182,86,0.92,0.92,10,5,1,0,34,32,0.780345,11,14,0.37,0.47,0.84,0.34,0.81,7.9,10.2,0.26,0.34,0.346,2.73,0.94,0.12,0.36,0.734,53,40,62,31,28,0.132,33,0.297,38,4,34,10,3,0,7,93,0.00607,56,0.866,54,53,20,36,3,1,0,205,0.256,0,0,0,0,0,0,0.0000,0,0,0.210526,0
4,198706,Luis Alberto,24,183,74,0.77,0.82,7,2,1,0,34,32,0.780345,11,14,0.37,0.47,0.84,0.34,0.81,7.9,10.2,0.26,0.34,0.346,2.73,0.94,0.12,0.36,0.734,53,40,62,31,28,0.132,33,0.297,38,4,34,10,3,0,7,93,0.00607,56,0.866,54,53,20,36,3,1,0,205,0.256,0,0,0,0,0,0,0.0000,0,0,0.210526,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,194894,Francesco Rossi,26,193,83,0.58,0.62,1,1,1,0,1,0,0.002047,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,1.000,0,0,0,0,0,0.000,0,0.000,0,0,0,0,0,0,3,0,0.00000,0,0.000,0,0,0,0,0,0,0,1,0.000,1,0,8,1,2,1,0.0050,0,0,0.315789,0
2746,224263,Jonjoe Kenny,20,176,67,0.57,0.72,2,1,1,0,19,17,0.464756,0,1,0.00,0.06,0.06,0.00,0.06,0.2,1.0,0.01,0.06,0.333,0.17,0.06,0.00,0.00,0.680,32,18,29,41,13,0.391,14,0.238,44,9,35,25,54,0,79,11,0.00412,8,0.924,11,7,17,4,0,1,1,178,0.646,0,0,0,0,0,0,0.0000,0,0,0.368421,0
2747,229910,Svante Ingelsson,19,189,83,0.56,0.72,4,1,0,1,7,3,0.094765,1,1,0.28,0.28,0.55,0.28,0.55,0.5,0.2,0.13,0.04,0.333,2.49,0.83,0.11,0.33,0.684,4,2,8,12,8,0.182,9,0.239,4,0,4,3,4,0,5,9,0.00571,5,0.755,4,11,4,3,1,0,0,32,0.556,0,0,0,0,0,0,0.0000,0,0,0.684211,0
2748,229880,Aaron Wan-Bissaka,19,183,72,0.55,0.71,4,1,1,0,7,7,0.183094,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.4,0.00,0.06,0.000,0.00,0.00,0.00,0.00,0.670,13,8,12,25,11,0.625,3,0.306,36,8,28,15,25,0,42,1,0.00727,9,0.931,5,6,2,3,0,0,0,57,0.500,0,0,0,0,0,0,0.0000,0,0,0.526316,0


In [28]:
#Dropping long_name from databases
datos_train1 = datos_train1.drop('long_name', axis = 1)
datos_train3 = datos_train3.drop('long_name', axis = 1)
datos_test = datos_test.drop('long_name', axis = 1)
datos_train1

,sofifa_id,age,height_cm,weight_kg,overall,potential,player_positions,international_reputation,right_foot,left_foot,games,games_starts,minutes,goals,assists,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,xa,xg_per90,xa_per90,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,passes_pct,crosses,passes_intercepted,passes_blocked,tackles,tackles_won,dribble_tackles_pct,dribbled_past,pressure_regain_pct,blocks,blocked_shots,blocked_passes,interceptions,clearances,errors,touches_def_pen_area,touches_att_pen_area,dribbles_completed_pct,players_dribbled_past,passes_received_pct,miscontrols,dispossessed,fouls,fouled,offsides,pens_won,pens_conceded,ball_recoveries,aerials_won_pct,games_gk,games_starts_gk,minutes_gk,goals_against_gk,shots_on_target_against,saves,save_pct,clean_sheets,pens_saved,LgRk,CL
0,20801,32,185,80,0.94,0.94,9,5,1,0,27,27,0.668909,26,5,1.02,0.20,1.22,0.90,1.10,24.1,4.3,0.95,0.17,0.403,6.92,2.79,0.13,0.32,0.797,44,19,29,5,3,0.100,9,0.253,14,5,9,6,15,0,19,245,0.00689,32,0.669,47,23,19,18,41,1,0,72,0.595,0,0,0,0,0,0,0.0000,0,0,0.105263,1
1,190871,25,175,68,0.92,0.94,9,5,1,0,20,20,0.525300,19,13,0.95,0.65,1.60,0.75,1.40,15.4,11.8,0.78,0.59,0.437,4.36,1.90,0.17,0.39,0.760,58,48,55,15,9,0.259,20,0.285,10,0,10,3,0,0,0,159,0.00624,127,0.886,64,58,24,103,10,2,0,110,0.500,0,0,0,0,0,0,0.0000,0,0,0.000000,1
2,167495,31,193,92,0.92,0.92,1,5,1,0,3,3,0.078678,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.852,0,1,0,0,0,0.000,0,0.000,0,0,0,0,1,0,67,1,0.00000,0,1.000,0,0,0,0,0,0,0,15,0.000,3,3,270,2,8,6,0.0075,2,0,0.000000,1
3,176580,30,182,86,0.92,0.92,10,5,1,0,34,32,0.780345,11,14,0.37,0.47,0.84,0.34,0.81,7.9,10.2,0.26,0.34,0.346,2.73,0.94,0.12,0.36,0.734,53,40,62,31,28,0.132,33,0.297,38,4,34,10,3,0,7,93,0.00607,56,0.866,54,53,20,36,3,1,0,205,0.256,0,0,0,0,0,0,0.0000,0,0,0.210526,0
4,198706,24,183,74,0.77,0.82,7,2,1,0,34,32,0.780345,11,14,0.37,0.47,0.84,0.34,0.81,7.9,10.2,0.26,0.34,0.346,2.73,0.94,0.12,0.36,0.734,53,40,62,31,28,0.132,33,0.297,38,4,34,10,3,0,7,93,0.00607,56,0.866,54,53,20,36,3,1,0,205,0.256,0,0,0,0,0,0,0.0000,0,0,0.210526,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,194894,26,193,83,0.58,0.62,1,1,1,0,1,0,0.002047,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,1.000,0,0,0,0,0,0.000,0,0.000,0,0,0,0,0,0,3,0,0.00000,0,0.000,0,0,0,0,0,0,0,1,0.000,1,0,8,1,2,1,0.0050,0,0,0.315789,0
2746,224263,20,176,67,0.57,0.72,2,1,1,0,19,17,0.464756,0,1,0.00,0.06,0.06,0.00,0.06,0.2,1.0,0.01,0.06,0.333,0.17,0.06,0.00,0.00,0.680,32,18,29,41,13,0.391,14,0.238,44,9,35,25,54,0,79,11,0.00412,8,0.924,11,7,17,4,0,1,1,178,0.646,0,0,0,0,0,0,0.0000,0,0,0.368421,0
2747,229910,19,189,83,0.56,0.72,4,1,0,1,7,3,0.094765,1,1,0.28,0.28,0.55,0.28,0.55,0.5,0.2,0.13,0.04,0.333,2.49,0.83,0.11,0.33,0.684,4,2,8,12,8,0.182,9,0.239,4,0,4,3,4,0,5,9,0.00571,5,0.755,4,11,4,3,1,0,0,32,0.556,0,0,0,0,0,0,0.0000,0,0,0.684211,0
2748,229880,19,183,72,0.55,0.71,4,1,1,0,7,7,0.183094,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.4,0.00,0.06,0.000,0.00,0.00,0.00,0.00,0.670,13,8,12,25,11,0.625,3,0.306,36,8,28,15,25,0,42,1,0.00727,9,0.931,5,6,2,3,0,0,0,57,0.500,0,0,0,0,0,0,0.0000,0,0,0.526316,0


In [29]:
datos_test

,sofifa_id,age,height_cm,weight_kg,overall,potential,player_positions,international_reputation,right_foot,left_foot,games,games_starts,minutes,goals,assists,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,xa,xg_per90,xa_per90,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,passes_pct,crosses,passes_intercepted,passes_blocked,tackles,tackles_won,dribble_tackles_pct,dribbled_past,pressure_regain_pct,blocks,blocked_shots,blocked_passes,interceptions,clearances,errors,touches_def_pen_area,touches_att_pen_area,dribbles_completed_pct,players_dribbled_past,passes_received_pct,miscontrols,dispossessed,fouls,fouled,offsides,pens_won,pens_conceded,ball_recoveries,aerials_won_pct,games_gk,games_starts_gk,minutes_gk,goals_against_gk,shots_on_target_against,saves,save_pct,clean_sheets,pens_saved,LgRk,CL
0,20801,34,187,83,0.93,0.93,10,5,1,0,33,33,0.852881,31,5,0.96,0.15,1.11,0.59,0.74,26.7,5.1,0.83,0.16,0.342,6.05,2.07,0.10,0.28,0.843,33,25,53,16,10,0.200,16,0.222,16,6,10,6,23,0,28,242,0.00655,60,0.780,69,50,25,58,31,2,0,112,0.493,0,0,0,0,0,0,0.00000,0,0,0.000000,1
1,190871,27,175,68,0.92,0.92,9,5,1,0,15,15,0.385200,13,6,0.89,0.41,1.30,0.61,1.02,12.7,5.9,0.87,0.40,0.484,4.37,2.12,0.14,0.29,0.790,28,24,24,14,7,0.185,22,0.288,10,0,10,6,1,0,2,109,0.00570,94,0.876,61,66,14,62,1,1,0,89,0.500,0,0,0,0,0,0,0.00000,0,0,0.000000,1
2,200389,26,188,87,0.91,0.93,1,3,1,0,38,38,0.992688,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.581,0,4,3,0,0,0.000,0,0.000,0,0,0,0,0,0,896,0,0.01000,1,0.991,1,0,0,7,0,0,0,148,0.000,38,38,3395,27,105,82,0.00771,17,0,0.105263,1
3,183277,28,175,74,0.91,0.91,9,4,1,0,16,14,0.314712,1,3,0.08,0.25,0.33,0.08,0.33,2.4,2.6,0.20,0.22,0.737,1.59,1.17,0.05,0.07,0.854,18,9,19,9,7,0.231,10,0.283,9,0,9,10,0,0,5,104,0.00621,44,0.831,24,36,12,50,2,3,0,75,0.556,0,0,0,0,0,0,0.00000,0,0,0.000000,1
4,192985,28,181,70,0.91,0.91,7,4,1,0,35,32,0.816028,13,20,0.42,0.64,1.06,0.35,1.00,7.6,18.4,0.24,0.59,0.281,3.10,0.87,0.11,0.41,0.746,143,61,96,44,28,0.190,47,0.319,31,2,29,11,19,1,30,164,0.00646,64,0.894,43,44,26,28,3,0,0,232,0.450,0,0,0,0,0,0,0.00000,0,0,0.052632,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2671,236867,20,185,80,0.56,0.67,1,1,1,0,4,3,0.092425,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.526,0,0,0,0,0,0.000,0,0.000,0,0,0,0,0,0,109,0,0.00000,0,1.000,0,0,0,0,0,0,0,16,0.000,4,3,316,6,18,12,0.00667,1,0,0.894737,0
2672,251292,22,192,84,0.56,0.64,1,1,1,0,17,17,0.447207,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.5,0.00,0.03,0.000,0.00,0.00,0.00,0.00,0.687,0,3,1,0,0,0.000,0,0.000,1,0,1,0,1,1,590,0,0.01000,3,0.997,0,0,1,1,0,0,1,112,0.000,17,17,1530,31,76,50,0.00618,2,1,0.842105,0
2673,235267,20,187,66,0.55,0.66,3,1,0,1,1,0,0.000000,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.000,0,0,0,0,0,0.000,0,0.000,0,0,0,0,0,0,0,0,0.00000,0,0.000,0,0,0,0,0,0,0,0,0.000,0,0,0,0,0,0,0.00000,0,0,1.000000,0
2674,244446,20,182,77,0.52,0.59,8,1,1,0,1,0,0.000585,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.500,0,1,0,0,0,0.000,0,0.000,0,0,0,0,0,0,0,0,0.00000,0,0.667,0,0,1,0,0,0,0,0,0.000,0,0,0,0,0,0,0.00000,0,0,0.894737,0


In [30]:
#Deleting extra rows in training set
#Sorting by sofifa_id

datos_train1.sort_values('sofifa_id', inplace=True,na_position='first')
datos_train3.sort_values('sofifa_id', inplace=True,na_position='first')
datos_test.sort_values('sofifa_id', inplace=True,na_position='first')

In [31]:
datos_train1

,sofifa_id,age,height_cm,weight_kg,overall,potential,player_positions,international_reputation,right_foot,left_foot,games,games_starts,minutes,goals,assists,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,xa,xg_per90,xa_per90,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,passes_pct,crosses,passes_intercepted,passes_blocked,tackles,tackles_won,dribble_tackles_pct,dribbled_past,pressure_regain_pct,blocks,blocked_shots,blocked_passes,interceptions,clearances,errors,touches_def_pen_area,touches_att_pen_area,dribbles_completed_pct,players_dribbled_past,passes_received_pct,miscontrols,dispossessed,fouls,fouled,offsides,pens_won,pens_conceded,ball_recoveries,aerials_won_pct,games_gk,games_starts_gk,minutes_gk,goals_against_gk,shots_on_target_against,saves,save_pct,clean_sheets,pens_saved,LgRk,CL
813,768,32,188,75,0.75,0.75,3,2,0,1,6,6,0.133372,0,0,0.00,0.00,0.00,0.00,0.00,0.1,0.1,0.02,0.01,0.000,0.39,0.00,0.00,0.00,0.732,1,4,4,7,4,0.000,4,0.288,8,3,5,7,24,0,34,4,0.01000,2,1.000,4,1,9,8,0,0,0,72,0.727,0,0,0,0,0,0,0.0000,0,0,0.631579,0
744,887,33,185,81,0.65,0.65,3,1,1,0,1,0,0.002632,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,1.000,0,0,0,0,0,0.000,0,0.000,0,0,0,0,0,0,0,0,0.00000,0,0.750,0,1,0,0,0,0,0,0,0.000,0,0,0,0,0,0,0.0000,0,0,0.631579,0
12,1179,39,191,94,0.89,0.89,1,4,1,0,21,21,0.544311,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.793,0,2,0,0,0,0.000,0,0.000,1,0,1,0,1,1,545,0,0.01000,1,0.993,0,0,2,4,0,0,1,86,0.000,21,21,1862,14,58,48,0.0081,11,1,0.000000,1
1974,3665,40,178,75,0.74,0.74,7,2,1,0,26,16,0.400702,2,2,0.13,0.13,0.26,0.13,0.26,1.4,3.0,0.09,0.20,0.250,1.05,0.26,0.13,0.50,0.771,15,23,28,27,16,0.116,38,0.228,24,3,21,18,18,0,23,33,0.00529,11,0.823,26,27,25,25,1,0,0,114,0.348,0,0,0,0,0,0,0.0000,0,0,0.947368,0
1582,5213,35,185,80,0.77,0.77,3,2,1,0,23,20,0.530564,0,0,0.00,0.00,0.00,0.00,0.00,0.7,0.0,0.03,0.00,0.250,0.20,0.05,0.00,0.00,0.849,0,1,4,19,7,0.500,7,0.248,45,26,19,11,119,0,161,6,0.00000,0,0.988,1,2,8,11,0,0,1,198,0.685,0,0,0,0,0,0,0.0000,0,0,0.631579,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2675,241049,20,176,72,0.63,0.75,2,1,1,0,2,1,0.021936,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.830,2,0,1,5,2,0.400,3,0.077,4,0,4,1,0,0,0,0,0.00000,0,0.825,1,1,1,1,0,0,0,10,0.000,0,0,0,0,0,0,0.0000,0,0,0.578947,0
907,241053,18,188,80,0.63,0.79,10,1,1,0,29,28,0.692893,5,2,0.19,0.08,0.27,0.15,0.23,5.1,2.2,0.19,0.08,0.341,1.67,0.57,0.09,0.27,0.678,24,37,41,71,60,0.313,46,0.244,50,8,42,45,19,0,42,68,0.00583,57,0.831,39,44,58,52,2,1,0,302,0.595,0,0,0,0,0,0,0.0000,0,0,0.842105,0
2677,241067,21,184,75,0.63,0.74,6,1,1,0,6,0,0.024861,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.04,0.01,0.000,1.05,0.00,0.00,0.00,0.735,0,2,2,7,4,0.200,4,0.244,2,0,2,2,0,0,0,2,0.01000,6,0.878,2,3,4,0,0,0,0,18,0.200,0,0,0,0,0,0,0.0000,0,0,0.894737,0
1213,241087,19,185,78,0.53,0.62,3,1,1,0,11,4,0.172858,0,0,0.00,0.00,0.00,0.00,0.00,0.2,0.5,0.04,0.09,0.500,0.30,0.15,0.00,0.00,0.836,11,3,4,9,3,0.250,6,0.360,10,0,10,3,25,0,35,6,0.00000,1,0.957,0,0,4,0,2,0,1,60,0.429,0,0,0,0,0,0,0.0000,0,0,0.210526,0


In [32]:
#Deleting extra rows in training set
#datos_train1 -- 2584 rows
#datos_train3 -- 2429 rows
#175 different rows

datos_train = pd.merge(datos_train1, datos_train3, on="sofifa_id")

datos_train

#datos_train_list1 = datos_train1.values.tolist()
#datos_train_list2 = datos_train3.values.tolist()
#datos_train.drop_duplicates()

#datos_train

#for i in range (len(datos_train_list2)):
#  for j in range (len(datos_train_list2)):
#     if str(datos_train_list1[j][0]) in str(datos_train_list2[i][0]):        
#          datos_train_list1[j][0] = datos_train_list2[i][0]

#for entries in datos_train_list2:
 #   if str(datos_train_list1[i][0]) in str(datos_train_list2[j][0]):        
 #         pandas.DataFrame.replace(cur_val, np.NaN)

#datos_train_list10[1][0]

,sofifa_id,age_x,height_cm_x,weight_kg_x,overall_x,potential_x,player_positions_x,international_reputation_x,right_foot_x,left_foot_x,games_x,games_starts_x,minutes_x,goals_x,assists_x,goals_per90_x,assists_per90_x,goals_assists_per90_x,goals_pens_per90_x,goals_assists_pens_per90_x,xg_x,xa_x,xg_per90_x,xa_per90_x,shots_on_target_pct_x,shots_total_per90_x,shots_on_target_per90_x,goals_per_shot_x,goals_per_shot_on_target_x,passes_pct_x,crosses_x,passes_intercepted_x,passes_blocked_x,tackles_x,tackles_won_x,dribble_tackles_pct_x,dribbled_past_x,pressure_regain_pct_x,blocks_x,blocked_shots_x,...,passes_pct_y,crosses_y,passes_intercepted_y,passes_blocked_y,tackles_y,tackles_won_y,dribble_tackles_pct_y,dribbled_past_y,pressure_regain_pct_y,blocks_y,blocked_shots_y,blocked_passes_y,interceptions_y,clearances_y,errors_y,touches_def_pen_area_y,touches_att_pen_area_y,dribbles_completed_pct_y,players_dribbled_past_y,passes_received_pct_y,miscontrols_y,dispossessed_y,fouls_y,fouled_y,offsides_y,pens_won_y,pens_conceded_y,ball_recoveries_y,aerials_won_pct_y,games_gk_y,games_starts_gk_y,minutes_gk_y,goals_against_gk_y,shots_on_target_against_y,saves_y,save_pct_y,clean_sheets_y,pens_saved_y,LgRk_y,CL_y
0,768,32,188,75,0.75,0.75,3,2,0,1,6,6,0.133372,0,0,0.00,0.00,0.00,0.00,0.00,0.1,0.1,0.02,0.01,0.000,0.39,0.00,0.00,0.00,0.732,1,4,4,7,4,0.000,4,0.288,8,3,...,0.00726,3.0,5.0,10.0,28.0,20.0,0.00484,8.0,0.00349,24.0,9.0,15.0,22.0,84.0,0.0,109.0,12.0,0.000075,2.0,0.00947,4.0,2.0,17.0,17.0,0.0,0.0,1.0,139.0,0.00609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.842105,0.0
1,1179,39,191,94,0.89,0.89,1,4,1,0,21,21,0.544311,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.793,0,2,0,0,0,0.000,0,0.000,1,0,...,0.00600,0.0,1.0,1.0,0.0,0.0,0.00000,1.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,367.0,0.0,0.000061,1.0,0.00847,0.0,0.0,1.0,1.0,0.0,0.0,0.0,78.0,0.00556,17.0,17.0,1530.0,18.0,61.0,47.0,0.0,6.0,1.0,0.000000,1.0
2,8205,34,181,77,0.73,0.73,6,1,1,0,25,23,0.589354,0,0,0.00,0.00,0.00,0.00,0.00,0.1,1.1,0.00,0.05,0.500,0.09,0.04,0.00,0.00,0.775,37,25,28,26,16,0.462,14,0.345,44,9,...,0.00732,13.0,15.0,13.0,27.0,19.0,0.00375,4.0,0.00327,10.0,2.0,8.0,7.0,41.0,1.0,65.0,4.0,0.000079,8.0,0.00951,3.0,1.0,4.0,9.0,3.0,0.0,0.0,97.0,0.00556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.842105,0.0
3,20775,33,175,67,0.83,0.83,4,3,1,0,18,15,0.394560,1,1,0.07,0.07,0.13,0.07,0.13,0.6,0.8,0.04,0.06,0.333,0.80,0.27,0.08,0.25,0.742,19,29,25,36,19,0.385,16,0.335,35,5,...,0.00748,44.0,29.0,45.0,48.0,32.0,0.00529,41.0,0.00198,52.0,2.0,50.0,31.0,10.0,0.0,21.0,82.0,0.000062,48.0,0.00941,69.0,42.0,17.0,27.0,2.0,0.0,0.0,199.0,0.00368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.368421,0.0
4,20801,32,185,80,0.94,0.94,9,5,1,0,27,27,0.668909,26,5,1.02,0.20,1.22,0.90,1.10,24.1,4.3,0.95,0.17,0.403,6.92,2.79,0.13,0.32,0.797,44,19,29,5,3,0.100,9,0.253,14,5,...,0.00832,59.0,28.0,31.0,10.0,8.0,0.00571,8.0,0.00325,9.0,3.0,6.0,9.0,29.0,0.0,37.0,220.0,0.000050,54.0,0.00974,62.0,39.0,20.0,50.0,28.0,0.0,0.0,115.0,0.00629,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1845,241010,23,186,74,0.59,0.63,3,1,1,0,28,26,0.661304,2,6,0.08,0.24,0.32,0.08,0.32,1.6,7.2,0.06,0.28,0.265,1.35,0.36,0.06,0.22,0.795,142,37,66,47,32,0.294,36,0.308,46,1,...,0.00765,59.0,34.0,43.0,37.0,17.0,0.00333,23.0,0.00182,33.0,2.0,31.0,12.0,23.0,0.0,41.0,57.0,0.000000,39.0,0.00977,26.0,15.0,19.0,20.0,5.0,0.0,0.0,150.0,0.00900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.105263,0.0
1846,241010,23,186,74,0.59,0.63,3,1,1,0,28,26,0.661304,2,6,0.08,0.24,0.32,0.08,0.32,1.6,7.2,0.06,0.28,0.265,1.35,0.36,0.06,0.22,0.795,142,37,66,47,32,0.294,36,0.308,46,1,...,0.00787,59.0,34.0,43.0,37.0,17.0,0.00000,23.0,0.00500,33.0,2.0,31

In [33]:
datos_train18 = datos_train
datos_train18.drop(datos_train18.iloc[:, 69:137], axis = 1, inplace = True)
datos_train18

,sofifa_id,age_x,height_cm_x,weight_kg_x,overall_x,potential_x,player_positions_x,international_reputation_x,right_foot_x,left_foot_x,games_x,games_starts_x,minutes_x,goals_x,assists_x,goals_per90_x,assists_per90_x,goals_assists_per90_x,goals_pens_per90_x,goals_assists_pens_per90_x,xg_x,xa_x,xg_per90_x,xa_per90_x,shots_on_target_pct_x,shots_total_per90_x,shots_on_target_per90_x,goals_per_shot_x,goals_per_shot_on_target_x,passes_pct_x,crosses_x,passes_intercepted_x,passes_blocked_x,tackles_x,tackles_won_x,dribble_tackles_pct_x,dribbled_past_x,pressure_regain_pct_x,blocks_x,blocked_shots_x,blocked_passes_x,interceptions_x,clearances_x,errors_x,touches_def_pen_area_x,touches_att_pen_area_x,dribbles_completed_pct_x,players_dribbled_past_x,passes_received_pct_x,miscontrols_x,dispossessed_x,fouls_x,fouled_x,offsides_x,pens_won_x,pens_conceded_x,ball_recoveries_x,aerials_won_pct_x,games_gk_x,games_starts_gk_x,minutes_gk_x,goals_against_gk_x,shots_on_target_against_x,saves_x,save_pct_x,clean_sheets_x,pens_saved_x,LgRk_x,CL_x
0,768,32,188,75,0.75,0.75,3,2,0,1,6,6,0.133372,0,0,0.00,0.00,0.00,0.00,0.00,0.1,0.1,0.02,0.01,0.000,0.39,0.00,0.00,0.00,0.732,1,4,4,7,4,0.000,4,0.288,8,3,5,7,24,0,34,4,0.01000,2,1.000,4,1,9,8,0,0,0,72,0.727,0,0,0,0,0,0,0.0000,0,0,0.631579,0
1,1179,39,191,94,0.89,0.89,1,4,1,0,21,21,0.544311,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.793,0,2,0,0,0,0.000,0,0.000,1,0,1,0,1,1,545,0,0.01000,1,0.993,0,0,2,4,0,0,1,86,0.000,21,21,1862,14,58,48,0.0081,11,1,0.000000,1
2,8205,34,181,77,0.73,0.73,6,1,1,0,25,23,0.589354,0,0,0.00,0.00,0.00,0.00,0.00,0.1,1.1,0.00,0.05,0.500,0.09,0.04,0.00,0.00,0.775,37,25,28,26,16,0.462,14,0.345,44,9,35,33,41,0,96,10,0.00565,14,0.952,11,6,19,9,0,0,1,204,0.541,0,0,0,0,0,0,0.0000,0,0,0.736842,0
3,20775,33,175,67,0.83,0.83,4,3,1,0,18,15,0.394560,1,1,0.07,0.07,0.13,0.07,0.13,0.6,0.8,0.04,0.06,0.333,0.80,0.27,0.08,0.25,0.742,19,29,25,36,19,0.385,16,0.335,35,5,30,27,69,0,68,14,0.00619,13,0.921,14,19,27,19,1,0,1,192,0.726,0,0,0,0,0,0,0.0000,0,0,0.263158,0
4,20801,32,185,80,0.94,0.94,9,5,1,0,27,27,0.668909,26,5,1.02,0.20,1.22,0.90,1.10,24.1,4.3,0.95,0.17,0.403,6.92,2.79,0.13,0.32,0.797,44,19,29,5,3,0.100,9,0.253,14,5,9,6,15,0,19,245,0.00689,32,0.669,47,23,19,18,41,1,0,72,0.595,0,0,0,0,0,0,0.0000,0,0,0.105263,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1845,241010,23,186,74,0.59,0.63,3,1,1,0,28,26,0.661304,2,6,0.08,0.24,0.32,0.08,0.32,1.6,7.2,0.06,0.28,0.265,1.35,0.36,0.06,0.22,0.795,142,37,66,47,32,0.294,36,0.308,46,1,45,26,8,0,282,34,0.00000,0,0.986,7,4,32,8,1,0,0,446,0.747,0,0,0,0,0,0,0.0000,0,0,0.105263,1
1846,241010,23,186,74,0.59,0.63,3,1,1,0,28,26,0.661304,2,6,0.08,0.24,0.32,0.08,0.32,1.6,7.2,0.06,0.28,0.265,1.35,0.36,0.06,0.22,0.795,142,37,66,47,32,0.294,36,0.308,46,1,45,26,8,0,282,34,0.00000,0,0.986,7,4,32,8,1,0,0,446,0.747,0,0,0,0,0,0,0.0000,0,0,0.105263,1
1847,241049,20,176,72,0.63,0.75,2,1,1,0,2,1,0.021936,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.830,2,0,1,5,2,0.400,3,0.077,4,0,4,1,0,0,0,0,0.00000,0,0.825,1,1,1,1,0,0,0,10,0.000,0,0,0,0,0,0,0.0000,0,0,0.578947,0
1848,241087,19,185,78,0.53,0.62,3,1,1,0,11,4,0.172858,0,0,0.00,0.00,0.00,0.00,0.00,0.2,0.5,0.04,0.09,0.500,0.30,0.15,0.00,0.00,0.836,11,3,4,9,3,0.250,6,0.360,10,0,10,3,25,0,35,6,0.00000,1,0.957,0,0,4,0,2,0,1,60,0.429,0,0,0,0,0,0,0.0000,0,0,0.210526,0


In [34]:
datos_train = pd.merge(datos_train1, datos_train3, on="sofifa_id")

datos_train

,sofifa_id,age_x,height_cm_x,weight_kg_x,overall_x,potential_x,player_positions_x,international_reputation_x,right_foot_x,left_foot_x,games_x,games_starts_x,minutes_x,goals_x,assists_x,goals_per90_x,assists_per90_x,goals_assists_per90_x,goals_pens_per90_x,goals_assists_pens_per90_x,xg_x,xa_x,xg_per90_x,xa_per90_x,shots_on_target_pct_x,shots_total_per90_x,shots_on_target_per90_x,goals_per_shot_x,goals_per_shot_on_target_x,passes_pct_x,crosses_x,passes_intercepted_x,passes_blocked_x,tackles_x,tackles_won_x,dribble_tackles_pct_x,dribbled_past_x,pressure_regain_pct_x,blocks_x,blocked_shots_x,...,passes_pct_y,crosses_y,passes_intercepted_y,passes_blocked_y,tackles_y,tackles_won_y,dribble_tackles_pct_y,dribbled_past_y,pressure_regain_pct_y,blocks_y,blocked_shots_y,blocked_passes_y,interceptions_y,clearances_y,errors_y,touches_def_pen_area_y,touches_att_pen_area_y,dribbles_completed_pct_y,players_dribbled_past_y,passes_received_pct_y,miscontrols_y,dispossessed_y,fouls_y,fouled_y,offsides_y,pens_won_y,pens_conceded_y,ball_recoveries_y,aerials_won_pct_y,games_gk_y,games_starts_gk_y,minutes_gk_y,goals_against_gk_y,shots_on_target_against_y,saves_y,save_pct_y,clean_sheets_y,pens_saved_y,LgRk_y,CL_y
0,768,32,188,75,0.75,0.75,3,2,0,1,6,6,0.133372,0,0,0.00,0.00,0.00,0.00,0.00,0.1,0.1,0.02,0.01,0.000,0.39,0.00,0.00,0.00,0.732,1,4,4,7,4,0.000,4,0.288,8,3,...,0.00726,3.0,5.0,10.0,28.0,20.0,0.00484,8.0,0.00349,24.0,9.0,15.0,22.0,84.0,0.0,109.0,12.0,0.000075,2.0,0.00947,4.0,2.0,17.0,17.0,0.0,0.0,1.0,139.0,0.00609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.842105,0.0
1,1179,39,191,94,0.89,0.89,1,4,1,0,21,21,0.544311,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.793,0,2,0,0,0,0.000,0,0.000,1,0,...,0.00600,0.0,1.0,1.0,0.0,0.0,0.00000,1.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,367.0,0.0,0.000061,1.0,0.00847,0.0,0.0,1.0,1.0,0.0,0.0,0.0,78.0,0.00556,17.0,17.0,1530.0,18.0,61.0,47.0,0.0,6.0,1.0,0.000000,1.0
2,8205,34,181,77,0.73,0.73,6,1,1,0,25,23,0.589354,0,0,0.00,0.00,0.00,0.00,0.00,0.1,1.1,0.00,0.05,0.500,0.09,0.04,0.00,0.00,0.775,37,25,28,26,16,0.462,14,0.345,44,9,...,0.00732,13.0,15.0,13.0,27.0,19.0,0.00375,4.0,0.00327,10.0,2.0,8.0,7.0,41.0,1.0,65.0,4.0,0.000079,8.0,0.00951,3.0,1.0,4.0,9.0,3.0,0.0,0.0,97.0,0.00556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.842105,0.0
3,20775,33,175,67,0.83,0.83,4,3,1,0,18,15,0.394560,1,1,0.07,0.07,0.13,0.07,0.13,0.6,0.8,0.04,0.06,0.333,0.80,0.27,0.08,0.25,0.742,19,29,25,36,19,0.385,16,0.335,35,5,...,0.00748,44.0,29.0,45.0,48.0,32.0,0.00529,41.0,0.00198,52.0,2.0,50.0,31.0,10.0,0.0,21.0,82.0,0.000062,48.0,0.00941,69.0,42.0,17.0,27.0,2.0,0.0,0.0,199.0,0.00368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.368421,0.0
4,20801,32,185,80,0.94,0.94,9,5,1,0,27,27,0.668909,26,5,1.02,0.20,1.22,0.90,1.10,24.1,4.3,0.95,0.17,0.403,6.92,2.79,0.13,0.32,0.797,44,19,29,5,3,0.100,9,0.253,14,5,...,0.00832,59.0,28.0,31.0,10.0,8.0,0.00571,8.0,0.00325,9.0,3.0,6.0,9.0,29.0,0.0,37.0,220.0,0.000050,54.0,0.00974,62.0,39.0,20.0,50.0,28.0,0.0,0.0,115.0,0.00629,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1845,241010,23,186,74,0.59,0.63,3,1,1,0,28,26,0.661304,2,6,0.08,0.24,0.32,0.08,0.32,1.6,7.2,0.06,0.28,0.265,1.35,0.36,0.06,0.22,0.795,142,37,66,47,32,0.294,36,0.308,46,1,...,0.00765,59.0,34.0,43.0,37.0,17.0,0.00333,23.0,0.00182,33.0,2.0,31.0,12.0,23.0,0.0,41.0,57.0,0.000000,39.0,0.00977,26.0,15.0,19.0,20.0,5.0,0.0,0.0,150.0,0.00900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.105263,0.0
1846,241010,23,186,74,0.59,0.63,3,1,1,0,28,26,0.661304,2,6,0.08,0.24,0.32,0.08,0.32,1.6,7.2,0.06,0.28,0.265,1.35,0.36,0.06,0.22,0.795,142,37,66,47,32,0.294,36,0.308,46,1,...,0.00787,59.0,34.0,43.0,37.0,17.0,0.00000,23.0,0.00500,33.0,2.0,31

In [35]:
datos_train19 = datos_train
datos_train19.drop(datos_train19.iloc[:, 1:69], axis = 1, inplace = True)
datos_train19

,sofifa_id,age_y,height_cm_y,weight_kg_y,overall_y,potential_y,player_positions_y,international_reputation_y,right_foot_y,left_foot_y,games_y,games_starts_y,minutes_y,goals_y,assists_y,goals_per90_y,assists_per90_y,goals_assists_per90_y,goals_pens_per90_y,goals_assists_pens_per90_y,xg_y,xa_y,xg_per90_y,xa_per90_y,shots_on_target_pct_y,shots_total_per90_y,shots_on_target_per90_y,goals_per_shot_y,goals_per_shot_on_target_y,passes_pct_y,crosses_y,passes_intercepted_y,passes_blocked_y,tackles_y,tackles_won_y,dribble_tackles_pct_y,dribbled_past_y,pressure_regain_pct_y,blocks_y,blocked_shots_y,blocked_passes_y,interceptions_y,clearances_y,errors_y,touches_def_pen_area_y,touches_att_pen_area_y,dribbles_completed_pct_y,players_dribbled_past_y,passes_received_pct_y,miscontrols_y,dispossessed_y,fouls_y,fouled_y,offsides_y,pens_won_y,pens_conceded_y,ball_recoveries_y,aerials_won_pct_y,games_gk_y,games_starts_gk_y,minutes_gk_y,goals_against_gk_y,shots_on_target_against_y,saves_y,save_pct_y,clean_sheets_y,pens_saved_y,LgRk_y,CL_y
0,768,33,188,75,0.75,0.75,3,2,0,1,17.0,14.0,0.319977,2.0,0.0,0.16,0.00,0.16,0.16,0.16,0.8,0.0,0.07,0.00,0.00143,0.82,0.16,0.20,1.00,0.00726,3.0,5.0,10.0,28.0,20.0,0.00484,8.0,0.00349,24.0,9.0,15.0,22.0,84.0,0.0,109.0,12.0,0.000075,2.0,0.00947,4.0,2.0,17.0,17.0,0.0,0.0,1.0,139.0,0.00609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.842105,0.0
1,1179,40,192,92,0.88,0.88,1,4,1,0,17.0,17.0,0.447207,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00000,0.00,0.00,0.00,0.00,0.00600,0.0,1.0,1.0,0.0,0.0,0.00000,1.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,367.0,0.0,0.000061,1.0,0.00847,0.0,0.0,1.0,1.0,0.0,0.0,0.0,78.0,0.00556,17.0,17.0,1530.0,18.0,61.0,47.0,0.0,6.0,1.0,0.000000,1.0
2,8205,35,181,75,0.71,0.71,6,1,1,0,14.0,14.0,0.330506,0.0,1.0,0.00,0.08,0.08,0.00,0.08,0.1,0.3,0.01,0.02,0.00000,0.16,0.00,0.00,0.00,0.00732,13.0,15.0,13.0,27.0,19.0,0.00375,4.0,0.00327,10.0,2.0,8.0,7.0,41.0,1.0,65.0,4.0,0.000079,8.0,0.00951,3.0,1.0,4.0,9.0,3.0,0.0,0.0,97.0,0.00556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.842105,0.0
3,20775,34,175,67,0.84,0.84,4,3,1,0,34.0,25.0,0.618602,1.0,3.0,0.04,0.13,0.17,0.04,0.17,2.6,3.2,0.11,0.14,0.00400,1.02,0.21,0.04,0.20,0.00748,44.0,29.0,45.0,48.0,32.0,0.00529,41.0,0.00198,52.0,2.0,50.0,31.0,10.0,0.0,21.0,82.0,0.000062,48.0,0.00941,69.0,42.0,17.0,27.0,2.0,0.0,0.0,199.0,0.00368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.368421,0.0
4,20801,33,187,83,0.94,0.94,10,5,1,0,31.0,30.0,0.785902,21.0,8.0,0.70,0.27,0.97,0.54,0.80,22.8,3.8,0.76,0.13,0.00500,5.69,2.08,0.09,0.26,0.00832,59.0,28.0,31.0,10.0,8.0,0.00571,8.0,0.00325,9.0,3.0,6.0,9.0,29.0,0.0,37.0,220.0,0.000050,54.0,0.00974,62.0,39.0,20.0,50.0,28.0,0.0,0.0,115.0,0.00629,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1845,241010,24,186,74,0.58,0.62,3,1,1,0,33.0,33.0,0.541679,2.0,2.0,0.10,0.10,0.19,0.10,0.19,1.5,3.0,0.07,0.15,0.00000,1.70,0.58,0.06,0.17,0.00765,59.0,34.0,43.0,37.0,17.0,0.00333,23.0,0.00182,33.0,2.0,31.0,12.0,23.0,0.0,41.0,57.0,0.000000,39.0,0.00977,26.0,15.0,19.0,20.0,5.0,0.0,0.0,150.0,0.00900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.105263,0.0
1846,241010,24,186,74,0.58,0.62,3,1,1,0,23.0,21.0,0.541679,2.0,2.0,0.10,0.10,0.19,0.10,0.19,1.5,3.0,0.07,0.15,0.01000,1.70,0.58,0.06,0.17,0.00787,59.0,34.0,43.0,37.0,17.0,0.00000,23.0,0.00500,33.0,2.0,31.0,12.0,23.0,0.0,41.0,57.0,0.000050,39.0,0.00907,26.0,15.0,19.0,20.0,5.0,0.0,0.0,150.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.105263,1.0
1847,241049,21,174,73,0.68,0.77,2,1,1,0,7.0,6.0,0.132787,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.1,0.5,0.02,0.10,0.00143,0.40,0.00,0.00,0.00,0.00851,5.0,9.0,15.0,17.0,5.0,0.00607,6.0,0.00315,9.0,1.0,8.0,6.0,14.0,1.0,24.0,4.0,0.000050,1.0,0.00949,9.0,5.0,5.0,5.0,0.0,0.0,0.0,47.0,0.00667,0.0,0.0,0.0,0.0,0.0,0.0

In [36]:
#Merging with datos_train dataframes to produce same number of rows

datos_train18test = pd.merge(datos_train18, datos_test, on="sofifa_id")
datos_train19test = pd.merge(datos_train19, datos_test, on="sofifa_id")

In [37]:
#Deleting additional columns from merge

datos_train18final = datos_train18test
datos_train18final.drop(datos_train18test.iloc[:,69:137], axis = 1, inplace = True)
datos_train18final

,sofifa_id,age_x,height_cm_x,weight_kg_x,overall_x,potential_x,player_positions_x,international_reputation_x,right_foot_x,left_foot_x,games_x,games_starts_x,minutes_x,goals_x,assists_x,goals_per90_x,assists_per90_x,goals_assists_per90_x,goals_pens_per90_x,goals_assists_pens_per90_x,xg_x,xa_x,xg_per90_x,xa_per90_x,shots_on_target_pct_x,shots_total_per90_x,shots_on_target_per90_x,goals_per_shot_x,goals_per_shot_on_target_x,passes_pct_x,crosses_x,passes_intercepted_x,passes_blocked_x,tackles_x,tackles_won_x,dribble_tackles_pct_x,dribbled_past_x,pressure_regain_pct_x,blocks_x,blocked_shots_x,blocked_passes_x,interceptions_x,clearances_x,errors_x,touches_def_pen_area_x,touches_att_pen_area_x,dribbles_completed_pct_x,players_dribbled_past_x,passes_received_pct_x,miscontrols_x,dispossessed_x,fouls_x,fouled_x,offsides_x,pens_won_x,pens_conceded_x,ball_recoveries_x,aerials_won_pct_x,games_gk_x,games_starts_gk_x,minutes_gk_x,goals_against_gk_x,shots_on_target_against_x,saves_x,save_pct_x,clean_sheets_x,pens_saved_x,LgRk_x,CL_x
0,768,32,188,75,0.75,0.75,3,2,0,1,6,6,0.133372,0,0,0.00,0.00,0.00,0.00,0.00,0.1,0.1,0.02,0.01,0.000,0.39,0.00,0.00,0.00,0.732,1,4,4,7,4,0.000,4,0.288,8,3,5,7,24,0,34,4,0.01000,2,1.000,4,1,9,8,0,0,0,72,0.727,0,0,0,0,0,0,0.0000,0,0,0.631579,0
1,1179,39,191,94,0.89,0.89,1,4,1,0,21,21,0.544311,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.793,0,2,0,0,0,0.000,0,0.000,1,0,1,0,1,1,545,0,0.01000,1,0.993,0,0,2,4,0,0,1,86,0.000,21,21,1862,14,58,48,0.0081,11,1,0.000000,1
2,20775,33,175,67,0.83,0.83,4,3,1,0,18,15,0.394560,1,1,0.07,0.07,0.13,0.07,0.13,0.6,0.8,0.04,0.06,0.333,0.80,0.27,0.08,0.25,0.742,19,29,25,36,19,0.385,16,0.335,35,5,30,27,69,0,68,14,0.00619,13,0.921,14,19,27,19,1,0,1,192,0.726,0,0,0,0,0,0,0.0000,0,0,0.263158,0
3,20801,32,185,80,0.94,0.94,9,5,1,0,27,27,0.668909,26,5,1.02,0.20,1.22,0.90,1.10,24.1,4.3,0.95,0.17,0.403,6.92,2.79,0.13,0.32,0.797,44,19,29,5,3,0.100,9,0.253,14,5,9,6,15,0,19,245,0.00689,32,0.669,47,23,19,18,41,1,0,72,0.595,0,0,0,0,0,0,0.0000,0,0,0.105263,1
4,22391,35,165,66,0.59,0.59,10,1,1,0,27,23,0.605440,9,1,0.39,0.04,0.43,0.35,0.39,7.4,4.2,0.32,0.18,0.353,2.22,0.78,0.16,0.44,0.829,17,17,37,30,21,0.267,22,0.219,17,0,17,8,2,0,2,93,0.00754,52,0.781,52,74,16,34,8,0,0,148,0.000,0,0,0,0,0,0,0.0000,0,0,0.421053,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509,241010,23,186,74,0.59,0.63,3,1,1,0,28,26,0.661304,2,6,0.08,0.24,0.32,0.08,0.32,1.6,7.2,0.06,0.28,0.265,1.35,0.36,0.06,0.22,0.795,142,37,66,47,32,0.294,36,0.308,46,1,45,26,8,0,282,34,0.00000,0,0.986,7,4,32,8,1,0,0,446,0.747,0,0,0,0,0,0,0.0000,0,0,0.105263,1
1510,241010,23,186,74,0.59,0.63,3,1,1,0,28,26,0.661304,2,6,0.08,0.24,0.32,0.08,0.32,1.6,7.2,0.06,0.28,0.265,1.35,0.36,0.06,0.22,0.795,142,37,66,47,32,0.294,36,0.308,46,1,45,26,8,0,282,34,0.00000,0,0.986,7,4,32,8,1,0,0,446,0.747,0,0,0,0,0,0,0.0000,0,0,0.105263,1
1511,241049,20,176,72,0.63,0.75,2,1,1,0,2,1,0.021936,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.830,2,0,1,5,2,0.400,3,0.077,4,0,4,1,0,0,0,0,0.00000,0,0.825,1,1,1,1,0,0,0,10,0.000,0,0,0,0,0,0,0.0000,0,0,0.578947,0
1512,241108,18,175,73,0.53,0.70,4,1,0,1,31,17,0.445159,4,2,0.24,0.12,0.35,0.24,0.35,4.3,1.5,0.25,0.09,0.357,3.31,1.18,0.07,0.20,0.786,31,19,25,19,12,0.478,12,0.299,15,1,14,14,7,0,8,68,0.00620,31,0.801,29,23,8,18,16,0,0,116,0.000,0,0,0,0,0,0,0.0000,0,0,0.210526,0


In [38]:
#Merging with datos_train dataframes to produce same number of rows

datos_train18test = pd.merge(datos_train18, datos_test, on="sofifa_id")
datos_train19test = pd.merge(datos_train19, datos_test, on="sofifa_id")

In [39]:
#Deleting additional columns from merge

datos_train19final = datos_train19test
datos_train19final.drop(datos_train19test.iloc[:, 69:137], axis = 1, inplace = True)
datos_train19final

,sofifa_id,age_y,height_cm_y,weight_kg_y,overall_y,potential_y,player_positions_y,international_reputation_y,right_foot_y,left_foot_y,games_y,games_starts_y,minutes_y,goals_y,assists_y,goals_per90_y,assists_per90_y,goals_assists_per90_y,goals_pens_per90_y,goals_assists_pens_per90_y,xg_y,xa_y,xg_per90_y,xa_per90_y,shots_on_target_pct_y,shots_total_per90_y,shots_on_target_per90_y,goals_per_shot_y,goals_per_shot_on_target_y,passes_pct_y,crosses_y,passes_intercepted_y,passes_blocked_y,tackles_y,tackles_won_y,dribble_tackles_pct_y,dribbled_past_y,pressure_regain_pct_y,blocks_y,blocked_shots_y,blocked_passes_y,interceptions_y,clearances_y,errors_y,touches_def_pen_area_y,touches_att_pen_area_y,dribbles_completed_pct_y,players_dribbled_past_y,passes_received_pct_y,miscontrols_y,dispossessed_y,fouls_y,fouled_y,offsides_y,pens_won_y,pens_conceded_y,ball_recoveries_y,aerials_won_pct_y,games_gk_y,games_starts_gk_y,minutes_gk_y,goals_against_gk_y,shots_on_target_against_y,saves_y,save_pct_y,clean_sheets_y,pens_saved_y,LgRk_y,CL_y
0,768,33,188,75,0.75,0.75,3,2,0,1,17.0,14.0,0.319977,2.0,0.0,0.16,0.00,0.16,0.16,0.16,0.8,0.0,0.07,0.00,0.00143,0.82,0.16,0.20,1.00,0.00726,3.0,5.0,10.0,28.0,20.0,0.00484,8.0,0.00349,24.0,9.0,15.0,22.0,84.0,0.0,109.0,12.0,0.000075,2.0,0.00947,4.0,2.0,17.0,17.0,0.0,0.0,1.0,139.0,0.00609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.842105,0.0
1,1179,40,192,92,0.88,0.88,1,4,1,0,17.0,17.0,0.447207,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00000,0.00,0.00,0.00,0.00,0.00600,0.0,1.0,1.0,0.0,0.0,0.00000,1.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,367.0,0.0,0.000061,1.0,0.00847,0.0,0.0,1.0,1.0,0.0,0.0,0.0,78.0,0.00556,17.0,17.0,1530.0,18.0,61.0,47.0,0.0,6.0,1.0,0.000000,1.0
2,20775,34,175,67,0.84,0.84,4,3,1,0,34.0,25.0,0.618602,1.0,3.0,0.04,0.13,0.17,0.04,0.17,2.6,3.2,0.11,0.14,0.00400,1.02,0.21,0.04,0.20,0.00748,44.0,29.0,45.0,48.0,32.0,0.00529,41.0,0.00198,52.0,2.0,50.0,31.0,10.0,0.0,21.0,82.0,0.000062,48.0,0.00941,69.0,42.0,17.0,27.0,2.0,0.0,0.0,199.0,0.00368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.368421,0.0
3,20801,33,187,83,0.94,0.94,10,5,1,0,31.0,30.0,0.785902,21.0,8.0,0.70,0.27,0.97,0.54,0.80,22.8,3.8,0.76,0.13,0.00500,5.69,2.08,0.09,0.26,0.00832,59.0,28.0,31.0,10.0,8.0,0.00571,8.0,0.00325,9.0,3.0,6.0,9.0,29.0,0.0,37.0,220.0,0.000050,54.0,0.00974,62.0,39.0,20.0,50.0,28.0,0.0,0.0,115.0,0.00629,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0
4,22391,36,165,66,0.57,0.57,10,1,1,0,13.0,5.0,0.158526,2.0,1.0,0.33,0.17,0.50,0.33,0.50,1.9,1.4,0.32,0.24,0.00333,1.82,0.66,0.18,0.50,0.00770,6.0,6.0,5.0,8.0,4.0,0.00000,6.0,0.00233,6.0,0.0,6.0,0.0,1.0,0.0,2.0,33.0,0.000060,11.0,0.00680,13.0,21.0,9.0,5.0,3.0,0.0,0.0,33.0,0.00256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.210526,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509,241010,24,186,74,0.58,0.62,3,1,1,0,23.0,21.0,0.541679,2.0,2.0,0.10,0.10,0.19,0.10,0.19,1.5,3.0,0.07,0.15,0.01000,1.70,0.58,0.06,0.17,0.00787,59.0,34.0,43.0,37.0,17.0,0.00000,23.0,0.00500,33.0,2.0,31.0,12.0,23.0,0.0,41.0,57.0,0.000050,39.0,0.00907,26.0,15.0,19.0,20.0,5.0,0.0,0.0,150.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.105263,1.0
1510,241010,24,186,74,0.58,0.62,3,1,1,0,23.0,21.0,0.541679,2.0,2.0,0.10,0.10,0.19,0.10,0.19,1.5,3.0,0.07,0.15,0.01000,1.70,0.58,0.06,0.17,0.00787,59.0,34.0,43.0,37.0,17.0,0.00000,23.0,0.00500,33.0,2.0,31.0,12.0,23.0,0.0,41.0,57.0,0.000050,39.0,0.00907,26.0,15.0,19.0,20.0,5.0,0.0,0.0,150.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.105263,1.0
1511,241049,21,174,73,0.68,0.77,2,1,1,0,7.0,6.0,0.132787,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.1,0.5,0.02,0.10,0.00143,0.40,0.00,0.00,0.00,0.00851,5.0,9.0,15.0,17.0,5.0,0.00607,6.0,0.00315,9.0,1.0,8.0,6.0,14.0,1.0,24.0,4.0,0.000050,1.0,0.00949,9.0,5.0,5.0,5.0,0.0,0.0,0.0,47.0,0.00667,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [40]:
#Merging with datos_train dataframes to produce same number of rows

datos_train18test = pd.merge(datos_train18, datos_test, on="sofifa_id")
datos_train19test = pd.merge(datos_train19, datos_test, on="sofifa_id")

In [41]:
#Deleting additional columns from merge

datos_test2 = datos_train18test
datos_test2.drop(datos_train18test.iloc[:, 1:69], axis = 1, inplace = True)
datos_test2

,sofifa_id,age,height_cm,weight_kg,overall,potential,player_positions,international_reputation,right_foot,left_foot,games,games_starts,minutes,goals,assists,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,xa,xg_per90,xa_per90,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,passes_pct,crosses,passes_intercepted,passes_blocked,tackles,tackles_won,dribble_tackles_pct,dribbled_past,pressure_regain_pct,blocks,blocked_shots,blocked_passes,interceptions,clearances,errors,touches_def_pen_area,touches_att_pen_area,dribbles_completed_pct,players_dribbled_past,passes_received_pct,miscontrols,dispossessed,fouls,fouled,offsides,pens_won,pens_conceded,ball_recoveries,aerials_won_pct,games_gk,games_starts_gk,minutes_gk,goals_against_gk,shots_on_target_against,saves,save_pct,clean_sheets,pens_saved,LgRk,CL
0,768,34,188,75,0.74,0.74,3,2,0,1,25,23,0.614507,1,0,0.04,0.00,0.04,0.04,0.04,1.1,0.1,0.05,0.00,0.250,0.86,0.21,0.05,0.20,0.797,5,9,13,33,18,0.560,11,0.363,29,16,13,24,186,1,174,28,0.00800,4,0.953,4,2,30,14,1,0,1,228,0.713,0,0,0,0,0,0,0.00000,0,0,0.105263,1
1,1179,41,192,92,0.83,0.83,1,4,1,0,9,9,0.236619,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.2,0.00,0.03,0.000,0.00,0.00,0.00,0.00,0.920,0,0,1,0,0,0.000,0,0.000,0,0,0,0,0,0,201,0,0.00000,0,0.980,0,0,0,1,0,0,0,35,0.000,9,9,810,10,32,23,0.00719,1,0,0.000000,1
2,20775,35,175,67,0.81,0.81,4,3,1,0,27,15,0.374086,3,2,0.21,0.14,0.35,0.21,0.35,1.1,1.9,0.08,0.13,0.333,0.84,0.28,0.25,0.75,0.743,30,15,31,25,14,0.150,34,0.255,30,2,28,15,8,0,10,50,0.00597,42,0.790,35,34,19,10,0,0,0,115,0.063,0,0,0,0,0,0,0.00000,0,0,0.578947,0
3,20801,34,187,83,0.93,0.93,10,5,1,0,33,33,0.852881,31,5,0.96,0.15,1.11,0.59,0.74,26.7,5.1,0.83,0.16,0.342,6.05,2.07,0.10,0.28,0.843,33,25,53,16,10,0.200,16,0.222,16,6,10,6,23,0,28,242,0.00655,60,0.780,69,50,25,58,31,2,0,112,0.493,0,0,0,0,0,0,0.00000,0,0,0.000000,1
4,22391,37,165,64,0.56,0.56,10,1,1,0,8,0,0.018426,0,0,0.00,0.00,0.00,0.00,0.00,0.5,0.2,0.75,0.32,0.375,11.25,4.22,0.00,0.00,0.771,1,3,2,0,0,0.000,1,0.333,2,0,2,0,0,0,0,5,0.01000,4,0.737,1,2,1,3,0,0,0,13,0.000,0,0,0,0,0,0,0.00000,0,0,0.157895,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509,241010,25,186,74,0.64,0.68,3,1,1,0,17,17,0.439310,0,0,0.00,0.00,0.00,0.00,0.00,0.5,0.0,0.03,0.00,0.250,0.48,0.12,0.00,0.00,0.871,1,8,9,32,16,0.500,10,0.380,24,14,10,17,112,1,152,17,0.00000,0,0.986,2,2,13,8,0,0,0,194,0.781,0,0,0,0,0,0,0.00000,0,0,0.315789,0
1510,241010,25,186,74,0.64,0.68,3,1,1,0,15,15,0.378766,1,2,0.07,0.14,0.21,0.07,0.21,0.5,2.1,0.03,0.14,0.353,1.18,0.42,0.06,0.17,0.824,52,16,39,20,12,0.303,23,0.328,21,1,20,15,21,0,46,45,0.00769,10,0.921,19,14,7,18,0,2,0,130,0.588,0,0,0,0,0,0,0.00000,0,0,0.000000,1
1511,241049,22,174,73,0.70,0.77,2,1,1,0,14,8,0.208541,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.4,0.00,0.05,1.000,0.13,0.13,0.00,0.00,0.769,22,8,15,18,9,0.500,12,0.330,14,0,14,13,30,0,24,18,0.00625,5,0.899,10,4,10,8,0,0,0,74,0.524,0,0,0,0,0,0,0.00000,0,0,0.263158,0
1512,241108,20,178,72,0.55,0.71,4,1,0,1,11,8,0.193039,1,1,0.14,0.14,0.27,0.14,0.27,2.8,0.8,0.38,0.11,0.200,7.14,1.43,0.00,0.00,0.737,10,6,12,1,1,0.000,0,0.333,1,0,1,0,0,0,5,40,0.00593,17,0.813,12,7,8,11,0,0,0,61,0.286,0,0,0,0,0,0,0.00000,0,0,0.157895,1


In [42]:
#Merging with datos_test_nuevo dataframes to produce same number of rows

datos_test21 = pd.merge(datos_test2, datos_test_nuevo, on="sofifa_id")
datos_test21.sort_values('sofifa_id', inplace=True,na_position='first')
datos_test21

,sofifa_id,age,height_cm,weight_kg,overall,potential,player_positions,international_reputation,right_foot,left_foot,games,games_starts,minutes,goals,assists,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,xa,xg_per90,xa_per90,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,passes_pct,crosses,passes_intercepted,passes_blocked,tackles,tackles_won,dribble_tackles_pct,dribbled_past,pressure_regain_pct,blocks,blocked_shots,blocked_passes,interceptions,clearances,errors,touches_def_pen_area,touches_att_pen_area,dribbles_completed_pct,players_dribbled_past,passes_received_pct,miscontrols,dispossessed,fouls,fouled,offsides,pens_won,pens_conceded,ball_recoveries,aerials_won_pct,games_gk,games_starts_gk,minutes_gk,goals_against_gk,shots_on_target_against,saves,save_pct,clean_sheets,pens_saved,LgRk,CL,Overall
0,1179,41,192,92,0.83,0.83,1,4,1,0,9,9,0.236619,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.2,0.00,0.03,0.000,0.00,0.00,0.00,0.00,0.920,0,0,1,0,0,0.000,0,0.000,0,0,0,0,0,0,201,0,0.00000,0,0.980,0,0,0,1,0,0,0,35,0.000,9,9,810,10,32,23,0.00719,1,0,0.000000,1,82
1,20775,35,175,67,0.81,0.81,4,3,1,0,27,15,0.374086,3,2,0.21,0.14,0.35,0.21,0.35,1.1,1.9,0.08,0.13,0.333,0.84,0.28,0.25,0.75,0.743,30,15,31,25,14,0.150,34,0.255,30,2,28,15,8,0,10,50,0.00597,42,0.790,35,34,19,10,0,0,0,115,0.063,0,0,0,0,0,0,0.00000,0,0,0.578947,0,79
2,20801,34,187,83,0.93,0.93,10,5,1,0,33,33,0.852881,31,5,0.96,0.15,1.11,0.59,0.74,26.7,5.1,0.83,0.16,0.342,6.05,2.07,0.10,0.28,0.843,33,25,53,16,10,0.200,16,0.222,16,6,10,6,23,0,28,242,0.00655,60,0.780,69,50,25,58,31,2,0,112,0.493,0,0,0,0,0,0,0.00000,0,0,0.000000,1,92
3,22391,37,165,64,0.56,0.56,10,1,1,0,8,0,0.018426,0,0,0.00,0.00,0.00,0.00,0.00,0.5,0.2,0.75,0.32,0.375,11.25,4.22,0.00,0.00,0.771,1,3,2,0,0,0.000,1,0.333,2,0,2,0,0,0,0,5,0.01000,4,0.737,1,2,1,3,0,0,0,13,0.000,0,0,0,0,0,0,0.00000,0,0,0.157895,1,56
4,49000,37,183,74,0.75,0.75,1,2,1,0,23,16,0.374086,2,1,0.14,0.07,0.21,0.14,0.21,0.7,1.1,0.05,0.08,0.583,0.84,0.49,0.17,0.29,0.866,9,15,25,55,36,0.340,33,0.255,26,3,23,5,16,0,23,17,0.00621,20,0.932,14,18,30,24,0,0,0,123,0.588,0,0,0,0,0,0,0.00000,0,0,0.315789,0,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,240976,21,172,69,0.75,0.82,2,1,0,1,17,17,0.439310,0,0,0.00,0.00,0.00,0.00,0.00,0.5,0.0,0.03,0.00,0.250,0.48,0.12,0.00,0.00,0.871,1,8,9,32,16,0.500,10,0.380,24,14,10,17,112,1,152,17,0.00000,0,0.986,2,2,13,8,0,0,0,194,0.781,0,0,0,0,0,0,0.00000,0,0,0.315789,0,75
1347,240976,21,172,69,0.75,0.82,2,1,0,1,17,17,0.439310,0,0,0.00,0.00,0.00,0.00,0.00,0.5,0.0,0.03,0.00,0.250,0.48,0.12,0.00,0.00,0.871,1,8,9,32,16,0.500,10,0.380,24,14,10,17,112,1,152,17,0.00000,0,0.986,2,2,13,8,0,0,0,194,0.781,0,0,0,0,0,0,0.00000,0,0,0.315789,0,75
1352,241049,22,174,73,0.70,0.77,2,1,1,0,14,8,0.208541,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.4,0.00,0.05,1.000,0.13,0.13,0.00,0.00,0.769,22,8,15,18,9,0.500,12,0.330,14,0,14,13,30,0,24,18,0.00625,5,0.899,10,4,10,8,0,0,0,74,0.524,0,0,0,0,0,0,0.00000,0,0,0.263158,0,71
1353,241108,20,178,72,0.55,0.71,4,1,0,1,11,8,0.193039,1,1,0.14,0.14,0.27,0.14,0.27,2.8,0.8,0.38,0.11,0.200,7.14,1.43,0.00,0.00,0.737,10,6,12,1,1,0.000,0,0.333,1,0,1,0,0,0,5,40,0.00593,17,0.813,12,7,8,11,0,0,0,61,0.286,0,0,0,0,0,0,0.00000,0,0,0.157895,1,55


In [43]:
#Deleting additional columns from merge

datos_test2_final = datos_test21['Overall']
datos_test2_final

0       82
1       79
2       92
3       56
4       74
        ..
1345    75
1347    75
1352    71
1353    55
1354    55
Name: Overall, Length: 1355, dtype: int64

In [44]:
#Merging with datos_test_nuevo dataframes to produce same number of rows

datos_test21 = pd.merge(datos_test_nuevo, datos_test2, on="sofifa_id")
datos_test21.sort_values('sofifa_id', inplace=True,na_position='first')
datos_test21

,sofifa_id,Overall,age,height_cm,weight_kg,overall,potential,player_positions,international_reputation,right_foot,left_foot,games,games_starts,minutes,goals,assists,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,xa,xg_per90,xa_per90,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,passes_pct,crosses,passes_intercepted,passes_blocked,tackles,tackles_won,dribble_tackles_pct,dribbled_past,pressure_regain_pct,blocks,blocked_shots,blocked_passes,interceptions,clearances,errors,touches_def_pen_area,touches_att_pen_area,dribbles_completed_pct,players_dribbled_past,passes_received_pct,miscontrols,dispossessed,fouls,fouled,offsides,pens_won,pens_conceded,ball_recoveries,aerials_won_pct,games_gk,games_starts_gk,minutes_gk,goals_against_gk,shots_on_target_against,saves,save_pct,clean_sheets,pens_saved,LgRk,CL
1283,1179,82,41,192,92,0.83,0.83,1,4,1,0,9,9,0.236619,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.2,0.00,0.03,0.000,0.00,0.00,0.00,0.00,0.920,0,0,1,0,0,0.000,0,0.000,0,0,0,0,0,0,201,0,0.00000,0,0.980,0,0,0,1,0,0,0,35,0.000,9,9,810,10,32,23,0.00719,1,0,0.000000,1
569,20775,79,35,175,67,0.81,0.81,4,3,1,0,27,15,0.374086,3,2,0.21,0.14,0.35,0.21,0.35,1.1,1.9,0.08,0.13,0.333,0.84,0.28,0.25,0.75,0.743,30,15,31,25,14,0.150,34,0.255,30,2,28,15,8,0,10,50,0.00597,42,0.790,35,34,19,10,0,0,0,115,0.063,0,0,0,0,0,0,0.00000,0,0,0.578947,0
14,20801,92,34,187,83,0.93,0.93,10,5,1,0,33,33,0.852881,31,5,0.96,0.15,1.11,0.59,0.74,26.7,5.1,0.83,0.16,0.342,6.05,2.07,0.10,0.28,0.843,33,25,53,16,10,0.200,16,0.222,16,6,10,6,23,0,28,242,0.00655,60,0.780,69,50,25,58,31,2,0,112,0.493,0,0,0,0,0,0,0.00000,0,0,0.000000,1
1213,22391,56,37,165,64,0.56,0.56,10,1,1,0,8,0,0.018426,0,0,0.00,0.00,0.00,0.00,0.00,0.5,0.2,0.75,0.32,0.375,11.25,4.22,0.00,0.00,0.771,1,3,2,0,0,0.000,1,0.333,2,0,2,0,0,0,0,5,0.01000,4,0.737,1,2,1,3,0,0,0,13,0.000,0,0,0,0,0,0,0.00000,0,0,0.157895,1
1285,49000,74,37,183,74,0.75,0.75,1,2,1,0,23,16,0.374086,2,1,0.14,0.07,0.21,0.14,0.21,0.7,1.1,0.05,0.08,0.583,0.84,0.49,0.17,0.29,0.866,9,15,25,55,36,0.340,33,0.255,26,3,23,5,16,0,23,17,0.00621,20,0.932,14,18,30,24,0,0,0,123,0.588,0,0,0,0,0,0,0.00000,0,0,0.315789,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,240976,75,21,172,69,0.75,0.82,2,1,0,1,17,17,0.439310,0,0,0.00,0.00,0.00,0.00,0.00,0.5,0.0,0.03,0.00,0.250,0.48,0.12,0.00,0.00,0.871,1,8,9,32,16,0.500,10,0.380,24,14,10,17,112,1,152,17,0.00000,0,0.986,2,2,13,8,0,0,0,194,0.781,0,0,0,0,0,0,0.00000,0,0,0.315789,0
346,240976,75,21,172,69,0.75,0.82,2,1,0,1,15,15,0.378766,1,2,0.07,0.14,0.21,0.07,0.21,0.5,2.1,0.03,0.14,0.353,1.18,0.42,0.06,0.17,0.824,52,16,39,20,12,0.303,23,0.328,21,1,20,15,21,0,46,45,0.00769,10,0.921,19,14,7,18,0,2,0,130,0.588,0,0,0,0,0,0,0.00000,0,0,0.000000,1
1100,241049,71,22,174,73,0.70,0.77,2,1,1,0,14,8,0.208541,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.4,0.00,0.05,1.000,0.13,0.13,0.00,0.00,0.769,22,8,15,18,9,0.500,12,0.330,14,0,14,13,30,0,24,18,0.00625,5,0.899,10,4,10,8,0,0,0,74,0.524,0,0,0,0,0,0,0.00000,0,0,0.263158,0
1258,241108,55,20,178,72,0.55,0.71,4,1,0,1,4,0,0.018134,0,0,0.00,0.00,0.00,0.00,0.00,0.4,0.0,0.50,0.03,0.391,3.13,1.23,0.04,0.11,0.782,0,0,1,17,10,0.167,15,0.322,7,0,7,4,1,0,0,8,0.00000,0,0.575,1,0,0,2,1,0,0,4,0.250,0,0,0,0,0,0,0.00000,0,0,0.473684,0


In [45]:
#Deleting additional columns from merge

datos_test_nuevo_final = datos_test21
datos_test_nuevo_final.drop('Overall', axis = 1, inplace = True)
datos_test_nuevo_final

,sofifa_id,age,height_cm,weight_kg,overall,potential,player_positions,international_reputation,right_foot,left_foot,games,games_starts,minutes,goals,assists,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,xa,xg_per90,xa_per90,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,passes_pct,crosses,passes_intercepted,passes_blocked,tackles,tackles_won,dribble_tackles_pct,dribbled_past,pressure_regain_pct,blocks,blocked_shots,blocked_passes,interceptions,clearances,errors,touches_def_pen_area,touches_att_pen_area,dribbles_completed_pct,players_dribbled_past,passes_received_pct,miscontrols,dispossessed,fouls,fouled,offsides,pens_won,pens_conceded,ball_recoveries,aerials_won_pct,games_gk,games_starts_gk,minutes_gk,goals_against_gk,shots_on_target_against,saves,save_pct,clean_sheets,pens_saved,LgRk,CL
1283,1179,41,192,92,0.83,0.83,1,4,1,0,9,9,0.236619,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.2,0.00,0.03,0.000,0.00,0.00,0.00,0.00,0.920,0,0,1,0,0,0.000,0,0.000,0,0,0,0,0,0,201,0,0.00000,0,0.980,0,0,0,1,0,0,0,35,0.000,9,9,810,10,32,23,0.00719,1,0,0.000000,1
569,20775,35,175,67,0.81,0.81,4,3,1,0,27,15,0.374086,3,2,0.21,0.14,0.35,0.21,0.35,1.1,1.9,0.08,0.13,0.333,0.84,0.28,0.25,0.75,0.743,30,15,31,25,14,0.150,34,0.255,30,2,28,15,8,0,10,50,0.00597,42,0.790,35,34,19,10,0,0,0,115,0.063,0,0,0,0,0,0,0.00000,0,0,0.578947,0
14,20801,34,187,83,0.93,0.93,10,5,1,0,33,33,0.852881,31,5,0.96,0.15,1.11,0.59,0.74,26.7,5.1,0.83,0.16,0.342,6.05,2.07,0.10,0.28,0.843,33,25,53,16,10,0.200,16,0.222,16,6,10,6,23,0,28,242,0.00655,60,0.780,69,50,25,58,31,2,0,112,0.493,0,0,0,0,0,0,0.00000,0,0,0.000000,1
1213,22391,37,165,64,0.56,0.56,10,1,1,0,8,0,0.018426,0,0,0.00,0.00,0.00,0.00,0.00,0.5,0.2,0.75,0.32,0.375,11.25,4.22,0.00,0.00,0.771,1,3,2,0,0,0.000,1,0.333,2,0,2,0,0,0,0,5,0.01000,4,0.737,1,2,1,3,0,0,0,13,0.000,0,0,0,0,0,0,0.00000,0,0,0.157895,1
1285,49000,37,183,74,0.75,0.75,1,2,1,0,23,16,0.374086,2,1,0.14,0.07,0.21,0.14,0.21,0.7,1.1,0.05,0.08,0.583,0.84,0.49,0.17,0.29,0.866,9,15,25,55,36,0.340,33,0.255,26,3,23,5,16,0,23,17,0.00621,20,0.932,14,18,30,24,0,0,0,123,0.588,0,0,0,0,0,0,0.00000,0,0,0.315789,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,240976,21,172,69,0.75,0.82,2,1,0,1,17,17,0.439310,0,0,0.00,0.00,0.00,0.00,0.00,0.5,0.0,0.03,0.00,0.250,0.48,0.12,0.00,0.00,0.871,1,8,9,32,16,0.500,10,0.380,24,14,10,17,112,1,152,17,0.00000,0,0.986,2,2,13,8,0,0,0,194,0.781,0,0,0,0,0,0,0.00000,0,0,0.315789,0
346,240976,21,172,69,0.75,0.82,2,1,0,1,15,15,0.378766,1,2,0.07,0.14,0.21,0.07,0.21,0.5,2.1,0.03,0.14,0.353,1.18,0.42,0.06,0.17,0.824,52,16,39,20,12,0.303,23,0.328,21,1,20,15,21,0,46,45,0.00769,10,0.921,19,14,7,18,0,2,0,130,0.588,0,0,0,0,0,0,0.00000,0,0,0.000000,1
1100,241049,22,174,73,0.70,0.77,2,1,1,0,14,8,0.208541,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.4,0.00,0.05,1.000,0.13,0.13,0.00,0.00,0.769,22,8,15,18,9,0.500,12,0.330,14,0,14,13,30,0,24,18,0.00625,5,0.899,10,4,10,8,0,0,0,74,0.524,0,0,0,0,0,0,0.00000,0,0,0.263158,0
1258,241108,20,178,72,0.55,0.71,4,1,0,1,4,0,0.018134,0,0,0.00,0.00,0.00,0.00,0.00,0.4,0.0,0.50,0.03,0.391,3.13,1.23,0.04,0.11,0.782,0,0,1,17,10,0.167,15,0.322,7,0,7,4,1,0,0,8,0.00000,0,0.575,1,0,0,2,1,0,0,4,0.250,0,0,0,0,0,0,0.00000,0,0,0.473684,0


In [46]:
# Dropping ID, no longer necessary

datos_train18final = datos_train18final.drop('sofifa_id', axis = 1)
datos_test2 = datos_test2.drop('sofifa_id', axis = 1)
datos_train19final = datos_train19final.drop('sofifa_id', axis = 1)
datos_test_nuevo_final = datos_test_nuevo_final.drop('sofifa_id', axis = 1)

In [47]:
# Separating overall and potential as "y" and rest of the table as "x"

datos_train_y = datos_test2['overall']
datos_train_y

datos_train_x = datos_train19final

#datos_train_y = new_matrix[:,:,4:6]

#datos_train_x = new_matrix
#datos_train_x = np.delete(datos_train_x, 4, axis=2)
#datos_train_x = np.delete(datos_train_x, 4, axis=2)
#datos_train_x[1][1][3]

#datos_train_x = np.delete(datos_train_x[:][1], 4, axis=2)
#datos_train_y = datos_train19[{'overall_y','potential_y'}]
#datos_train_x = datos_train19.drop(['overall_y', 'potential_y'], axis = 1)

In [48]:
# Transforming dataframes in arrays

datos_train_x = datos_train_x.astype('float32')
datos_train_y = np.asarray(datos_train_y).astype('float32')
datos_test2 = np.asarray(datos_test2).astype('float32')

In [49]:
# Separating data in training and validation

train_dfX,val_dfX,train_dfY, val_dfY = train_test_split(datos_train_x,datos_train_y, test_size=0.1, random_state=True)

In [50]:
#LSTM layers work on 3D data with the following structure (nb_sequence, nb_timestep, nb_feature).

#   nb_sequence corresponds to the total number of sequences in your dataset (or to the batch size if you are using mini-batch learning).
#   nb_timestep corresponds to the size of your sequences.
#   nb_feature corresponds to number of features describing each of your timesteps.


In [61]:
# Model
iii = layers.Input(shape=(68,)) #input 

d1 = layers.Dense(32, activation="sigmoid", input_dim=68)(iii)
d2 = layers.Dense(16, activation="sigmoid")(d1)
final = layers.Dense(1, activation="sigmoid")(d2)

modelo = keras.Model(inputs=iii, outputs=final)
modelo.summary()
opt = tf.keras.optimizers.Adam(lr=1e-3, decay=1e-5)
modelo.compile(opt, tf.keras.losses.MeanSquaredError(), metrics=['mse','mae'])
h = modelo.fit(train_dfX, train_dfY, epochs=300, batch_size = 32, validation_data=(val_dfX, val_dfY)) 

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 68)]              0         
                                                                 
 dense_3 (Dense)             (None, 32)                2208      
                                                                 
 dense_4 (Dense)             (None, 16)                528       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 2,753
Trainable params: 2,753
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/300
43/43 [==============================] - 2s 18ms/step - loss: 0.0596 - mse: 0.0596 - mae: 0.2177 - val_loss: 0.0091 - val_mse: 0.0091 - val_mae: 0.0771
Epoch 2/300
43/43 [==============================] - 0s 5ms/step - loss: 0.0049 - mse: 0.0049 - mae: 0.0544 - val_loss: 0.0034 - val_mse: 0.0034 - val_mae: 0.0456
Epoch 3/300
43/43 [==============================] - 0s 6ms/step - loss: 0.0035 - mse: 0.0035 - mae: 0.0453 - val_loss: 0.0030 - val_mse: 0.0030 - val_mae: 0.0431
Epoch 4/300
43/43 [==============================] - 0s 6ms/step - loss: 0.0033 - mse: 0.0033 - mae: 0.0441 - val_loss: 0.0029 - val_mse: 0.0029 - val_mae: 0.0425
Epoch 5/300
43/43 [==============================] - 0s 5ms/step - loss: 0.0032 - mse: 0.0032 - mae: 0.0433 - val_loss: 0.0028 - val_mse: 0.0028 - val_mae: 0.0418
Epoch 6/300
43/43 [==============================] - 0s 5ms/step - loss: 0.0031 - mse: 0.0031 - mae: 0.0427 - val_loss: 0.0028 - val_mse: 0.0028 - val_mae: 0.0416
Epoch 7/300
43/43 [==

In [62]:
X_test = datos_test_nuevo_final
y_test = datos_test2_final
y_pred = modelo.predict(X_test)

# Compute the root-mean-square
rms = np.sqrt(mean_squared_error(y_test, y_pred))
print(rms)

#predicciones = np.where(predicciones > 0.5, 1, 0) # prediction > 0.5 is 1, else 0
#dataframe with predictions
#submission = pd.DataFrame(predicciones,columns=['Survived'])

74.90546370767457


In [63]:
y_test2 = pd.DataFrame(y_test)
y_test2

,Overall
0,82
1,79
2,92
3,56
4,74
...,...
1345,75
1347,75
1352,71
1353,55


In [64]:
y_pred2 = pd.DataFrame(y_pred)
y_pred2 = y_pred2.div(0.01)
y_pred2

,0
0,81.755669
1,78.517281
2,86.561798
3,66.482834
4,73.060944
...,...
1350,79.559731
1351,76.898186
1352,74.292091
1353,70.228073


In [55]:
y_pred2.columns = ['Overall']
y_dif = pd.DataFrame(columns=['real_Overall','predicted_Overall','Diff'])
y_dif['real_Overall'] = y_test2['Overall']
y_dif['predicted_Overall'] = y_pred2['Overall']
y_dif['Diff'] = (y_dif['real_Overall']-y_dif['predicted_Overall'])
y_dif.loc[[0]]

,real_Overall,predicted_Overall,Diff
0,82,81.691048,0.308952


In [56]:
y_dif

,real_Overall,predicted_Overall,Diff
0,82,81.691048,0.308952
1,79,81.065170,-2.065170
2,92,87.530876,4.469124
3,56,75.984467,-19.984467
4,74,82.129883,-8.129883
...,...,...,...
1345,75,77.590309,-2.590309
1347,75,77.590309,-2.590309
1352,71,73.740028,-2.740028
1353,55,71.661507,-16.661507


In [57]:
datos_test21

,sofifa_id,age,height_cm,weight_kg,overall,potential,player_positions,international_reputation,right_foot,left_foot,games,games_starts,minutes,goals,assists,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg,xa,xg_per90,xa_per90,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,passes_pct,crosses,passes_intercepted,passes_blocked,tackles,tackles_won,dribble_tackles_pct,dribbled_past,pressure_regain_pct,blocks,blocked_shots,blocked_passes,interceptions,clearances,errors,touches_def_pen_area,touches_att_pen_area,dribbles_completed_pct,players_dribbled_past,passes_received_pct,miscontrols,dispossessed,fouls,fouled,offsides,pens_won,pens_conceded,ball_recoveries,aerials_won_pct,games_gk,games_starts_gk,minutes_gk,goals_against_gk,shots_on_target_against,saves,save_pct,clean_sheets,pens_saved,LgRk,CL
1283,1179,41,192,92,0.83,0.83,1,4,1,0,9,9,0.236619,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.2,0.00,0.03,0.000,0.00,0.00,0.00,0.00,0.920,0,0,1,0,0,0.000,0,0.000,0,0,0,0,0,0,201,0,0.00000,0,0.980,0,0,0,1,0,0,0,35,0.000,9,9,810,10,32,23,0.00719,1,0,0.000000,1
569,20775,35,175,67,0.81,0.81,4,3,1,0,27,15,0.374086,3,2,0.21,0.14,0.35,0.21,0.35,1.1,1.9,0.08,0.13,0.333,0.84,0.28,0.25,0.75,0.743,30,15,31,25,14,0.150,34,0.255,30,2,28,15,8,0,10,50,0.00597,42,0.790,35,34,19,10,0,0,0,115,0.063,0,0,0,0,0,0,0.00000,0,0,0.578947,0
14,20801,34,187,83,0.93,0.93,10,5,1,0,33,33,0.852881,31,5,0.96,0.15,1.11,0.59,0.74,26.7,5.1,0.83,0.16,0.342,6.05,2.07,0.10,0.28,0.843,33,25,53,16,10,0.200,16,0.222,16,6,10,6,23,0,28,242,0.00655,60,0.780,69,50,25,58,31,2,0,112,0.493,0,0,0,0,0,0,0.00000,0,0,0.000000,1
1213,22391,37,165,64,0.56,0.56,10,1,1,0,8,0,0.018426,0,0,0.00,0.00,0.00,0.00,0.00,0.5,0.2,0.75,0.32,0.375,11.25,4.22,0.00,0.00,0.771,1,3,2,0,0,0.000,1,0.333,2,0,2,0,0,0,0,5,0.01000,4,0.737,1,2,1,3,0,0,0,13,0.000,0,0,0,0,0,0,0.00000,0,0,0.157895,1
1285,49000,37,183,74,0.75,0.75,1,2,1,0,23,16,0.374086,2,1,0.14,0.07,0.21,0.14,0.21,0.7,1.1,0.05,0.08,0.583,0.84,0.49,0.17,0.29,0.866,9,15,25,55,36,0.340,33,0.255,26,3,23,5,16,0,23,17,0.00621,20,0.932,14,18,30,24,0,0,0,123,0.588,0,0,0,0,0,0,0.00000,0,0,0.315789,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,240976,21,172,69,0.75,0.82,2,1,0,1,17,17,0.439310,0,0,0.00,0.00,0.00,0.00,0.00,0.5,0.0,0.03,0.00,0.250,0.48,0.12,0.00,0.00,0.871,1,8,9,32,16,0.500,10,0.380,24,14,10,17,112,1,152,17,0.00000,0,0.986,2,2,13,8,0,0,0,194,0.781,0,0,0,0,0,0,0.00000,0,0,0.315789,0
346,240976,21,172,69,0.75,0.82,2,1,0,1,15,15,0.378766,1,2,0.07,0.14,0.21,0.07,0.21,0.5,2.1,0.03,0.14,0.353,1.18,0.42,0.06,0.17,0.824,52,16,39,20,12,0.303,23,0.328,21,1,20,15,21,0,46,45,0.00769,10,0.921,19,14,7,18,0,2,0,130,0.588,0,0,0,0,0,0,0.00000,0,0,0.000000,1
1100,241049,22,174,73,0.70,0.77,2,1,1,0,14,8,0.208541,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.4,0.00,0.05,1.000,0.13,0.13,0.00,0.00,0.769,22,8,15,18,9,0.500,12,0.330,14,0,14,13,30,0,24,18,0.00625,5,0.899,10,4,10,8,0,0,0,74,0.524,0,0,0,0,0,0,0.00000,0,0,0.263158,0
1258,241108,20,178,72,0.55,0.71,4,1,0,1,4,0,0.018134,0,0,0.00,0.00,0.00,0.00,0.00,0.4,0.0,0.50,0.03,0.391,3.13,1.23,0.04,0.11,0.782,0,0,1,17,10,0.167,15,0.322,7,0,7,4,1,0,0,8,0.00000,0,0.575,1,0,0,2,1,0,0,4,0.250,0,0,0,0,0,0,0.00000,0,0,0.473684,0


In [67]:
datos_test21 = datos_test21.reset_index(drop=True)
y_dif['sofifa_id'] = datos_test21['sofifa_id']
y_dif

,real_Overall,predicted_Overall,Diff,sofifa_id
0,82,81.691048,0.308952,1179
1,79,81.065170,-2.065170,20775
2,92,87.530876,4.469124,20801
3,56,75.984467,-19.984467,22391
4,74,82.129883,-8.129883,49000
...,...,...,...,...
1345,75,77.590309,-2.590309,240976
1347,75,77.590309,-2.590309,240976
1352,71,73.740028,-2.740028,241049
1353,55,71.661507,-16.661507,241108


In [68]:
#Searching players by sofifa_id (235805 -- Federico Chiesa)

player = np.where(y_dif['sofifa_id'] == 235805)
player

(array([1274]),)

In [69]:
y_dif.iloc[[1274]]

,real_Overall,predicted_Overall,Diff,sofifa_id
1274,78,81.261818,-3.261818,235805


In [70]:
#modelo.save('modelo_fifa.h5') #saving model as .h5